# SMA Strategy Analysis System
This notebook contains all the code organized in cells for running the complete SMA strategy analysis.
Run each cell in sequence to perform the full analysis.

In [ ]:
# Global configuration
underlyings = [
    "AD", "BO", "BP", "BRN", "C", "CC", "CD", "CL", "CT", "DX", 
    "EC", "EMD", "ES", "FC", "FDAX", "FESX", "FGBL", "FGBM", "FGBX", 
    "FV", "GC", "HG", "HO", "JY", "KC", "KW", "LC", "LH", "LJ", 
    "LZ", "MME", "MP1", "NE1", "NG", "NK", "NQ", "PA", "PL", 
    "RB", "RTY", "S", "SB", "SF", "SI", "SM", "TEN", "TY", 
    "TU", "UB", "ULS", "US", "VX", "W", "WBS"
]

## 1. Configuration (input.py)
Paste the contents of input.py here

In [ ]:
# Configuration file for SMA optimization strategy using genetic algorithm
# Ticker symbol for the asset will be set dynamically
TICKER = None  # This will be set before each run

# Date range for analysis
START_DATE = '2014-01-01'
END_DATE = '2025-01-01'

# SMA range parameters (for genetic algorithm bounds)
SMA_MIN = 10
SMA_MAX = 300
SMA_STEP = 5  # Kept for compatibility with original code, not used in GA

# Data splitting ratio (percentage of data used for in-sample testing)
TRAIN_TEST_SPLIT = 0.7

# Results output file
RESULTS_FILE = 'sma_all_results.txt'

# ATR-based position sizing parameters
ATR_PERIOD = 30          # Period for ATR calculation (days)
TRADING_CAPITAL = 6000   # Capital allocation for position sizing

# Genetic Algorithm parameters
POPULATION_SIZE = 2000    # Number of individuals in population
NUM_GENERATIONS = 20     # Number of generations to evolve
HALL_OF_FAME_SIZE = 100000   #didelis skaicius kad visus paimtu
CROSSOVER_PROB = 0.5     # Probability of crossover
MUTATION_PROB = 0.3      # Probability of mutation
RANDOM_SEED = 42         # Random seed for reproducibility

# Clustering parameters (kept for compatibility)
MIN_TRADES = 10              # Minimum number of trades to consider in clustering
MAX_TRADES = 2000            # Maximum number of trades to consider in clustering
MIN_ELEMENTS_PER_CLUSTER = 20  # Minimum elements required for a valid cluster
DEFAULT_NUM_CLUSTERS = 50     # Default number of clusters if not specified by user

## 2. Data Reading Utilities (read_ts.py)
Paste the contents of read_ts.py here

In [ ]:

from multiprocessing import Pool
import struct
import os
import glob
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List

class mdata:
    tick_size = None
    big_point_value = None
    country = None
    exchange = None
    symbol = None
    description = None
    interval_type = None
    interval_span = None
    time_zone = None
    session = None
    data = None

    def __repr__(self):
        return str((self.symbol, self.interval_type, self.interval_span, self.data.shape))

def read_ts_ohlcv_dat_one(fname) -> mdata:
    try:
        f = open(fname, "rb")
        d = read_ts_ohlcv_dat_one_stream(f)
    finally:
        f.close()
    return d

def read_ts_ohlcv_dat_one_stream(byte_stream) -> mdata:
    def read_string(f):
        sz = struct.unpack('i', f.read(4))[0]
        s = f.read(sz).decode('ascii')
        return s
    d = mdata()
    try:
        # f = open(fname, "rb")
        f = byte_stream
        (ones, type_format) = struct.unpack('ii', f.read(8))
        if (ones != 1111111111):
            print("format not supported, must be 1111111111")
            return None
        if (type_format != 3):
            print("type_format not supported, must be 3")
            return None
        d.tick_size = struct.unpack('d', f.read(8))[0]
        d.big_point_value = struct.unpack('d', f.read(8))[0]
        d.country = read_string(f)
        d.exchange = read_string(f)
        d.symbol = read_string(f)
        d.description = read_string(f)
        d.interval_type = read_string(f)
        d.interval_span = struct.unpack('i', f.read(4))[0]
        d.time_zone = read_string(f)
        d.session = read_string(f)

        dt = np.dtype([('date', 'f8'), ('open', 'f8'), ('high', 'f8'), ('low', 'f8'), ('close', 'f8'), ('volume', 'f8')])
        #hist2 = np.fromfile(f, dtype=dt)
        z = f.read()
        #hist2 = np.frombuffer(z, dtype=dt)
        #hist2 = pd.DataFrame.from_records(hist2)
        data = pd.DataFrame.from_records(np.frombuffer(z, dtype=dt))
    finally:
        #f.close() #no need to close - -will be closed outside
        pass

    arr = ((data['date']-25569)*24*60*60).round(0).astype(np.int64)*1000000000
    #25569 - difference between datenum('2088-02-25') - datenum('2018-02-23')
    # .round(0) - to remove millisecond error
    # *1000000000 - to convert to native type of nanoseconds
    z2 = pd.to_datetime(arr)
    data.insert(0, 'datetime', z2)
    del data['date']
    d.data = data
    return d

def read_ts_ohlcv_dat(fnames) -> List[mdata]:
    r = []
    for name in glob.glob(fnames):
        #print('loading ', name)
        z = read_ts_ohlcv_dat_one(name)
        r.append(z)
    #print('done')
    return r

class spnl:
    data = None
    fname = None
    symbol = None
    dir = None
    strat = None
    session = None
    trade_count = None
    bar_size = None
    param = None
    symbol_root = None
    sidb = None
    date = None
    pnl = None


########################################################################################################################
def read_ts_pnl_dat_one(fname)->spnl:
    """

    :param fname: string with file name or bytes with hist2
    :return:
    """

    if type(fname) is bytes:
        f = fname
        (ones, type_format) = struct.unpack('ii', f[:8])
        dt = np.dtype([('date', 'f4'), ('pnl', 'f4'), ('pos', 'f4'), ('trade', 'f4')])
        data = np.frombuffer(f[8:], dtype=dt)
        additional_info = ['a','a','a','a','a','a']
        sida = 'a_a_a_a_a_a'
    elif type(fname) is str:
        try:
            f = open(fname, "rb")
            (ones, type_format) = struct.unpack('ii', f.read(8))
            dt = np.dtype([('date', 'f4'), ('pnl', 'f4'), ('pos', 'f4'), ('trade', 'f4')])
            data = np.fromfile(f, dtype=dt)
            #hist2 = pd.DataFrame.from_records(hist2)
            additional_info = os.path.basename(fname).split('_')
            sida = fname.split('\\')[-1].replace('.dat','')
        finally:
            f.close()
    else:
        f = fname
        fname = os.path.basename(f.name)
        (ones, type_format) = struct.unpack('ii', f.read(8))
        dt = np.dtype([('date', 'f4'), ('pnl', 'f4'), ('pos', 'f4'), ('trade', 'f4')])
        data = np.frombuffer(f.read(), dtype=dt)
        additional_info = os.path.basename(fname).split('_')
        sida = fname.split('\\')[-1].replace('.dat','')



    #arr = np.floor(hist2['date'] - 25569).astype(np.int64) * 24 * 60 * 60 * 1000000000
    #z2 = pd.to_datetime(arr)
    #hist2.insert(0, 'datetime', z2)
    #del hist2['date']


    d = spnl()

    #d.date = pd.to_datetime(np.floor(hist2['date'] - 25569).astype(np.int64) * 24 * 60 * 60 * 1000000000).values
    #hist2['date'] =
    d.date = np.array(np.floor(data['date'] - 25569).astype(np.int64) * 24 * 60 * 60 * 1000000000, dtype='datetime64[ns]')
    d.pnl = data['pnl']

    #d.hist2 = hist2
    #d.date = hist2.datetime
    #d.pnl = hist2.pnl
    d.fname = fname
    d.symbol = additional_info[0]
    d.dir = additional_info[1]
    d.strat = additional_info[2]
    d.session = additional_info[3]
    d.trade_count = data['trade'].max()
    d.bar_size = additional_info[4]
    d.param = additional_info[5]
    d.symbol_root = additional_info[0]
    d.sida = sida
    k = d.sida.split('_')
    d.sidb = '_'.join(k[0:1] + k[2:-1])
    d.pos = data['pos']
    d.trade = data['trade']

    return d
########################################################################################################################
def read_ts_pnl_dat_one_with_entry_price(fname)->spnl:
    """

    :param fname: string with file name or bytes with hist2
    :return:
    """

    if type(fname) is bytes:
        f = fname
        (ones, type_format) = struct.unpack('ii', f[:8])
        dt = np.dtype([('date', 'f8'), ('pnl', 'f8'), ('pos', 'f8'), ('trade', 'f8'), ('entry_price', 'f8'), ('empty', 'f8')])
        data = np.frombuffer(f[8:], dtype=dt)
        additional_info = ['a','a','a','a','a','a']
        sida = 'a_a_a_a_a_a'
    elif type(fname) is str:
        try:
            f = open(fname, "rb")
            (ones, type_format) = struct.unpack('ii', f.read(8))
            dt = np.dtype([('date', 'f8'), ('pnl', 'f8'), ('pos', 'f8'), ('trade', 'f8'), ('entry_price', 'f8'), ('empty', 'f8')])
            data = np.fromfile(f, dtype=dt)
            #hist2 = pd.DataFrame.from_records(hist2)
            additional_info = os.path.basename(fname).split('_')
            sida = fname.split('\\')[-1].replace('.dat','')
        finally:
            f.close()
    else:
        f = fname
        fname = os.path.basename(f.name)
        (ones, type_format) = struct.unpack('ii', f.read(8))
        dt = np.dtype([('date', 'f4'), ('pnl', 'f4'), ('pos', 'f4'), ('trade', 'f4')])
        data = np.frombuffer(f.read(), dtype=dt)
        additional_info = os.path.basename(fname).split('_')
        sida = fname.split('\\')[-1].replace('.dat','')



    #arr = np.floor(hist2['date'] - 25569).astype(np.int64) * 24 * 60 * 60 * 1000000000
    #z2 = pd.to_datetime(arr)
    #hist2.insert(0, 'datetime', z2)
    #del hist2['date']


    d = spnl()

    #d.date = pd.to_datetime(np.floor(hist2['date'] - 25569).astype(np.int64) * 24 * 60 * 60 * 1000000000).values
    #hist2['date'] =
    d.date = np.array(np.floor(data['date'] - 25569).astype(np.int64) * 24 * 60 * 60 * 1000000000, dtype='datetime64[ns]')
    d.pnl = data['pnl']

    #d.hist2 = hist2
    #d.date = hist2.datetime
    #d.pnl = hist2.pnl
    d.fname = fname
    d.symbol = additional_info[0]
    d.dir = additional_info[1]
    d.strat = additional_info[2]
    d.session = additional_info[3]
    d.trade_count = data['trade'].max()
    d.bar_size = additional_info[4]
    d.param = additional_info[5]
    d.symbol_root = additional_info[0]
    d.sida = sida
    k = d.sida.split('_')
    d.sidb = '_'.join(k[0:1] + k[2:-1])
    d.pos = data['pos']
    d.trade = data['trade']
    d.entry_price = data['entry_price']

    return d
########################################################################################################################
def read_ts_pnl_dat(fnames):
    print('loading ', fnames)
    r = []
    counter = 0
    for name in glob.glob(fnames):
        #print('loading ', name)
        try:
            z = read_ts_pnl_dat_one(name)
            r.append(z)
            # print('done')
        except:
            print("cannot read :", name)
        else:
            counter += 1
    print(f"loaded files: {counter}")
    return r
########################################################################################################################
def read_ts_pnl_dat_multicore(fnames):
    #print(f'load started from {fnames}', end =" ")

    import threading
    import time

    files = glob.glob(fnames)

    #print(f', files: {len(files)}', end =" ")

    time_start = time.time()

    def read_one_file(one_file, results, index):
        results[index] = read_ts_pnl_dat_one(one_file)

    thread_list = [None] * len(files)
    results = [None] * len(files)
    for i in range(len(files)):
        thread_list[i] = threading.Thread(target=read_one_file, args=(files[i], results, i))
        thread_list[i].start()

    for o in thread_list:
        o.join()

    #print(f"done. elapsed: {(time.time() - time_start):7} seconds")
    return results
###############################################################
def read_ts_pnl_dat_multicore_with_entry_price(fnames):
    #print(f'load started from {fnames}', end =" ")

    import threading
    import time

    files = glob.glob(fnames)

    #print(f', files: {len(files)}', end =" ")

    time_start = time.time()

    def read_one_file(one_file, results, index):
        results[index] = read_ts_pnl_dat_one_with_entry_price(one_file)

    thread_list = [None] * len(files)
    results = [None] * len(files)
    for i in range(len(files)):
        thread_list[i] = threading.Thread(target=read_one_file, args=(files[i], results, i))
        thread_list[i].start()

    for o in thread_list:
        o.join()

    #print(f"done. elapsed: {(time.time() - time_start):7} seconds")
    return results
###############################################################
def read_ts_pnl_dat_multicore_old(fnames, number_of_threads=16):
    """
    plase use this form: if __name__ == '__main__':
    :param fnames:
    :return:
    """
    print('load started from ' + fnames)
    files = glob.glob(fnames)
    ilgis = len(files)
    chunksize = ilgis // number_of_threads
    agents = number_of_threads
    pool = Pool(agents)
    r = pool.map(read_ts_pnl_dat_one, files, chunksize)
    pool.close()
    print('done')
    return r
###############################################################
def read_ts_pnl_csv_one(fname: str) -> pd.DataFrame:
    a = pd.read_csv(fname, delimiter=',', index_col=0, parse_dates=True, dayfirst=False, header=None,
                    names=['pnl', 'c', 'd', 'e'])
    a.index = np.array(a.index, dtype='datetime64[D]')
    a.index.name = 'date'
    a.fname = fname
    a.sida = fname.split('\\')[-1].replace('.CSV', '')
    k = a.sida.split('_')
    a.sidb = '_'.join(k[0:1] + k[2:-1])
    a.symbol = k[0]
    return a
########################################################################################################################
def read_ts_pnl_csv(fnames: "") -> List[pd.DataFrame]: # type: ignore
    r = []
    for name in glob.glob(fnames):
        print('loading ', name)
        z = read_ts_pnl_csv_one(name)
        r.append(z)
    print('done')
    return r
########################################################################################################################
def allign_numpy(d) -> Tuple[np.ndarray, np.ndarray]:
    all_date = np.unique(np.concatenate([t.date for t in d]))
    all_date = np.array(all_date, dtype='datetime64[D]')
    all_pnl_matrix = np.zeros((len(d), len(all_date)), dtype='float32')
    i = 0
    for q in d:
        idx = np.in1d(all_date, q.date, assume_unique=True)
        all_pnl_matrix[i, idx] = q.pnl
        i = i + 1

    return all_date, all_pnl_matrix
########################################################################################################################
def allign_pandas(d) -> Tuple[np.ndarray, pd.DataFrame]:
    all_date = np.unique(np.concatenate([t.date for t in d]))
    all_date = np.array(all_date, dtype='datetime64[D]')
    df1 = pd.DataFrame(0.0, index=all_date, columns=[a.sidb for a in d])
    for q in d:
        df1.iloc[np.in1d(all_date, q.date, assume_unique=True), df1.columns.get_loc(q.sidb)] = q.pnl

    return all_date, df1
########################################################################################################################
def mavg_old_and_slow(d,p):
    r = np.zeros(len(d))
    r[p - 1] = d[:p].sum()
    for i in range(p, len(d)):
        r[i] = r[i - 1] - d[i - p] + d[i]
    return r / p
########################################################################################################################
def mavg(d, period):
    return np.concatenate([np.zeros((period-1)), np.convolve(d, np.ones((period,))/period, mode='valid')])
########################################################################################################################
def emavg(d : np.ndarray,period: int) -> np.ndarray:
    if period == 1:
        return d
    r = np.zeros(len(d))
    alpha = 2 / (period + 1)  # weight for ema
    # first exponential average is a first price
    r[0] = d[1]
    for j in range(1, len(d)):
        r[j] = r[j - 1] + alpha * (d[j] - r[j - 1])
    return r
########################################################################################################################
def smooth(data_ : np.ndarray, period : int) -> np.ndarray:
    #r = np.zeros(len(data_))
    r = data = data_
    #hist2: np.ndarray = data_
    for i in range(0,period):
        r[0] = data[0]
        r[-1] = data[-1]
        r[1:-1] = (data[:-2] + data[1:-1] + data[2:])/3
        data = r
    return data
########################################################################################################################
def remove_bad_is(data, oos_date):

    if len(data) < 10000:
        print(f"too few hist2: {len(data)} - will not delete")
        return data

    bad_res = []
    for i in range(len(data)):
        a1 = data[i]
        idx_oos_date = np.argmax(a1.date >= oos_date)
        if a1.pnl[:idx_oos_date].sum() <= 0:
            bad_res.append(i)

    print(f"bad ones: {len(bad_res)} will be deleted")

    del_percent = len(bad_res) / len(data) * 100
    if del_percent > 90:
        print(f"too many deletes: {del_percent}% - will not delete")
        return data

    #delete
    import os
    for i in bad_res:
        try:
            os.remove(data[i].fname)
        except OSError:
            pass

    #remove from list
    a3 = np.array(data)
    a4 = np.delete(a3, bad_res)
    return a4
########################################################################################################################



## 3. SMA Strategy Implementation (SMA_Strategy.py)
Paste the contents of SMA_Strategy.py here

In [ ]:
import pandas as pd
import numpy as np
import time
from read_ts import mavg  # Add import for mavg function

class SMAStrategy:
    """
    Vectorized SMA (Simple Moving Average) trading strategy with ATR-based position sizing
    
    Features:
    - Basic SMA crossover entry signals (long when short SMA crosses above long SMA, short when it crosses below)
    - Pure long/short positions based on crossover direction
    - Position size based on ATR volatility
    - Slippage modeling for realistic execution
    - P&L calculated in absolute dollar terms using big point value
    - Fully vectorized implementation for efficiency
    """

    def __init__(self, short_sma, long_sma, big_point_value, slippage=0, capital=6000, atr_period=50):
        """
        Initialize the SMA strategy with specific parameters

        Parameters:
        short_sma (int): Short SMA period in days
        long_sma (int): Long SMA period in days
        big_point_value (int): Contract big point value for calculating dollar P&L
        slippage (float): Slippage in price units added/subtracted from execution price
        capital (float): The capital allocation for position sizing
        atr_period (int): Period for ATR calculation for position sizing
        """
        self.short_sma = short_sma
        self.long_sma = long_sma
        self.big_point_value = big_point_value
        self.slippage = slippage
        self.capital = capital
        self.atr_period = atr_period

    def calculate_atr(self, data, period=None):
        """
        Calculate Average True Range (ATR)
        
        Parameters:
        data: DataFrame with OHLC data
        period: ATR calculation period, defaults to self.atr_period if None
        
        Returns:
        Series: ATR values
        """
        if period is None:
            period = self.atr_period
            
        # Calculate True Range
        # True High is the maximum of today's high and yesterday's close
        high = data['High'].copy()
        prev_close = data['Close'].shift(1)
        true_high = pd.DataFrame({'high': high, 'prev_close': prev_close}).max(axis=1)
        
        # True Low is the minimum of today's low and yesterday's close
        low = data['Low'].copy()
        true_low = pd.DataFrame({'low': low, 'prev_close': prev_close}).min(axis=1)
        
        # True Range = True High - True Low
        true_range = true_high - true_low
        
        # ATR is the moving average of True Range
        atr = true_range.rolling(window=period).mean()
        
        return atr

    def apply_strategy(self, data, strategy_name="Strategy"):
        """
        Apply the simple SMA crossover strategy to the price data using vectorized operations
        with ATR-based position sizing
        """
        # Create a copy of the DataFrame to avoid modifying the original
        sim_data = data.copy()
        
        # Calculate SMAs using the efficient mavg function from read_ts
        sim_data[f'SMA_Short_{strategy_name}'] = mavg(sim_data['Close'].values, self.short_sma)
        sim_data[f'SMA_Long_{strategy_name}'] = mavg(sim_data['Close'].values, self.long_sma)
        
        # Calculate ATR for position sizing
        sim_data[f'ATR_{strategy_name}'] = self.calculate_atr(sim_data, self.atr_period)
        
        # Calculate position size based on ATR
        sim_data[f'Position_Size_{strategy_name}'] = np.round(
            self.capital / (sim_data[f'ATR_{strategy_name}'] * self.big_point_value) + 0.5
        )
        
        # Determine position direction
        sim_data[f'Position_Dir_{strategy_name}'] = np.where(
            sim_data[f'SMA_Short_{strategy_name}'] > sim_data[f'SMA_Long_{strategy_name}'], 1, -1
        )
        
        # Fill NaN values at the beginning
        sim_data[f'Position_Dir_{strategy_name}'] = sim_data[f'Position_Dir_{strategy_name}'].fillna(0)
        
        # Identify position changes
        sim_data[f'Position_Change_{strategy_name}'] = sim_data[f'Position_Dir_{strategy_name}'].diff() != 0
        
        # Calculate P&L
        market_pnl = sim_data['Close'].diff() * self.big_point_value
        sim_data[f'Market_PnL_{strategy_name}'] = market_pnl
        
        # Strategy P&L
        sim_data[f'Daily_PnL_{strategy_name}'] = (
            market_pnl * 
            sim_data[f'Position_Dir_{strategy_name}'].shift(1) * 
            sim_data[f'Position_Size_{strategy_name}'].shift(1)
        )
        
        # Apply slippage at position changes
        position_changed = sim_data[f'Position_Change_{strategy_name}']
        sim_data.loc[position_changed, f'Daily_PnL_{strategy_name}'] -= (
            self.slippage * sim_data[f'Position_Size_{strategy_name}'][position_changed]
        )
        
        # Replace NaN values in first row
        sim_data[f'Daily_PnL_{strategy_name}'] = sim_data[f'Daily_PnL_{strategy_name}'].fillna(0)
        
        # Calculate cumulative P&L
        sim_data[f'Cumulative_PnL_{strategy_name}'] = sim_data[f'Daily_PnL_{strategy_name}'].cumsum()
        
        return sim_data

    def optimize(self, data, sma_range, train_test_split=0.7, results_file=None, warm_up_idx=None):
        """
        Find the optimal SMA parameters and record all simulations using vectorized operations

        Parameters:
        data: DataFrame with market data
        sma_range: Range of SMA periods to test
        train_test_split: Portion of data to use for in-sample testing
        results_file: Path to save simulation results
        warm_up_idx: Index to trim warm-up period (if provided)

        Returns:
        best_sma_params: Tuple with (short_sma, long_sma)
        best_sharpe: Best Sharpe ratio found
        best_trades: Number of trades with best parameters
        all_results: List of tuples with all simulation results
        """
        # Initialize variables to track the best performance
        best_sharpe = -np.inf  # Start with negative infinity to ensure any valid Sharpe ratio will be better
        best_sma = (0, 0)  # Tuple to store the best (short_sma, long_sma) combination
        best_trades = 0  # Number of trades with the best parameters
        best_sim_data = None  # Store the simulation data for the best parameters

        # Create a list to store all simulation results
        all_results = []

        # Create the output file for all simulations and write the header
        if results_file:
            with open(results_file, 'w') as f:
                f.write("short_SMA,long_SMA,trades,sharpe_ratio\n")

        # Count total combinations to test
        total_combinations = sum(1 for a, b in [(s, l) for s in sma_range for l in sma_range] if a < b)
        completed = 0

        print(f"Running {total_combinations} simulations...")

        # Track individual simulation times
        total_sim_time = 0
        
        # Iterate through all possible combinations of short and long SMA periods
        for short_sma in sma_range:
            for long_sma in sma_range:
                # Skip invalid combinations where short SMA is not actually shorter than long SMA
                if short_sma >= long_sma:
                    continue

                # Start timing this simulation
                sim_start_time = time.time()
                
                # Save original parameters
                orig_short_sma = self.short_sma
                orig_long_sma = self.long_sma

                # Set new parameters for this simulation
                self.short_sma = short_sma
                self.long_sma = long_sma

                # Apply strategy to the data
                sim_data = self.apply_strategy(data.copy(), strategy_name="Sim")

                # Trim warm-up period if provided
                if warm_up_idx is not None:
                    # Create an explicit copy to avoid the SettingWithCopyWarning
                    sim_data_eval = sim_data.iloc[warm_up_idx:].copy()
                    # Recalculate the split index based on the trimmed data length
                    split_index = int(len(sim_data_eval) * train_test_split)
                else:
                    # Since we're already working with a copy, no need to make another
                    sim_data_eval = sim_data
                    # Calculate split index for in-sample/out-of-sample
                    split_index = int(len(sim_data_eval) * train_test_split)

                # Count trades by identifying position changes
                trade_entries = sim_data_eval['Position_Change_Sim']
                trade_count = trade_entries.sum()

                # Calculate daily returns - using loc to avoid SettingWithCopyWarning
                sim_data_eval.loc[:, 'Daily_Returns'] = sim_data_eval['Daily_PnL_Sim']
                
                # Calculate Sharpe ratio using only in-sample data (on dollar returns)
                in_sample_returns = sim_data_eval['Daily_Returns'].iloc[:split_index]

                # Skip if there are no returns or all returns are 0
                if len(in_sample_returns.dropna()) == 0 or in_sample_returns.std() == 0:
                    sharpe_ratio = 0
                else:
                    sharpe_ratio = in_sample_returns.mean() / in_sample_returns.std() * np.sqrt(252)  # Annualized

                # Append the results to our file
                if results_file:
                    with open(results_file, 'a') as f:
                        f.write(f"{short_sma},{long_sma},{trade_count},{sharpe_ratio:.6f}\n")

                # Store the results
                result = (short_sma, long_sma, trade_count, sharpe_ratio)
                all_results.append(result)

                # Update best parameters if current combination performs better
                if sharpe_ratio > best_sharpe:
                    best_sharpe = sharpe_ratio
                    best_sma = (short_sma, long_sma)
                    best_trades = trade_count
                    best_sim_data = sim_data_eval.copy()  # Store for verification

                # Restore original parameters
                self.short_sma = orig_short_sma
                self.long_sma = orig_long_sma

                # End timing for this simulation
                sim_end_time = time.time()
                sim_time = sim_end_time - sim_start_time
                total_sim_time += sim_time
                
                # Update progress
                completed += 1
                if completed % 100 == 0 or completed == total_combinations:
                    avg_sim_time = total_sim_time / completed
                    est_remaining = avg_sim_time * (total_combinations - completed)
                    print(
                        f"Progress: {completed}/{total_combinations} simulations completed ({(completed / total_combinations * 100):.1f}%)"
                        f" - Avg sim time: {avg_sim_time:.4f}s - Est. remaining: {est_remaining:.1f}s")

        # Verify the calculation (for debugging)
        if best_sim_data is not None:
            print("\n--- OPTIMIZATION SHARPE VERIFICATION ---")
            
            # Calculate metrics on the best sim data
            verify_split_idx = int(len(best_sim_data) * train_test_split)
            verify_returns = best_sim_data['Daily_Returns'].iloc[:verify_split_idx]
            
            if verify_returns.std() > 0:
                verify_sharpe = verify_returns.mean() / verify_returns.std() * np.sqrt(252)
                print(f"Optimization best Sharpe = {best_sharpe:.6f}")
                print(f"Verification Sharpe = {verify_sharpe:.6f}")
                print(f"Data points used: {len(best_sim_data)}")
                print(f"In-sample data points: {len(verify_returns)}")
            else:
                print("Cannot verify Sharpe (std = 0)")

        # Return the optimal SMA parameters, corresponding Sharpe ratio, and all results
        return best_sma, best_sharpe, best_trades, all_results

    def calculate_performance_metrics(self, data, strategy_name="Strategy", train_test_split=0.7):
        """
        Calculate detailed performance metrics for the strategy

        Parameters:
        data: DataFrame with strategy results
        strategy_name: Name suffix for the strategy columns
        train_test_split: Portion of data used for in-sample testing

        Returns:
        dict: Dictionary with performance metrics
        """
        # Calculate split index for in-sample/out-of-sample
        split_index = int(len(data) * train_test_split)
        split_date = data.index[split_index]

        # Extract daily P&L data
        daily_pnl = data[f'Daily_PnL_{strategy_name}']
        
        # Split returns for performance comparison
        returns_in_sample = daily_pnl.iloc[:split_index]
        returns_out_sample = daily_pnl.iloc[split_index:]

        # Calculate separate Sharpe ratios on dollar P&L (annualized)
        sharpe_in_sample = returns_in_sample.mean() / returns_in_sample.std() * np.sqrt(
            252) if returns_in_sample.std() > 0 else 0
        sharpe_out_sample = returns_out_sample.mean() / returns_out_sample.std() * np.sqrt(
            252) if returns_out_sample.std() > 0 else 0
        sharpe_full = daily_pnl.mean() / daily_pnl.std() * np.sqrt(252) if daily_pnl.std() > 0 else 0

        # Print detailed Sharpe ratio calculation info for verification
        print("\n--- PERFORMANCE METRICS SHARPE VERIFICATION ---")
        print(f"In-sample Sharpe = {sharpe_in_sample:.6f}")
        print(f"Data points used: {len(data)}")
        print(f"In-sample data points: {len(returns_in_sample)}")
        print(f"Mean: {returns_in_sample.mean():.6f}, Std: {returns_in_sample.std():.6f}")

        # Calculate trade counts using position changes
        position_changes = data[f'Position_Change_{strategy_name}']
        
        total_trades = position_changes.sum()
        in_sample_trades = position_changes.iloc[:split_index].sum()
        out_sample_trades = position_changes.iloc[split_index:].sum()

        # Calculate max drawdown in dollar terms
        pnl_series = data[f'Cumulative_PnL_{strategy_name}']
        # Use loc to avoid SettingWithCopyWarning
        data.loc[:, 'Peak'] = pnl_series.cummax()
        data.loc[:, 'Drawdown_Dollars'] = pnl_series - data['Peak']
        max_drawdown_dollars = data['Drawdown_Dollars'].min()

        # Calculate profit/loss metrics
        total_pnl = data[f'Cumulative_PnL_{strategy_name}'].iloc[-1]

        # Calculate in-sample and out-of-sample P&L
        in_sample_pnl = data[f'Daily_PnL_{strategy_name}'].iloc[:split_index].sum()
        out_sample_pnl = data[f'Daily_PnL_{strategy_name}'].iloc[split_index:].sum()

        # Calculate average position size
        avg_position_size = data[f'Position_Size_{strategy_name}'].mean()
        max_position_size = data[f'Position_Size_{strategy_name}'].max()

        # Assemble the results in a dictionary
        metrics = {
            'split_date': split_date,
            'total_pnl': total_pnl,
            'sharpe_full': sharpe_full,
            'sharpe_in_sample': sharpe_in_sample,
            'sharpe_out_sample': sharpe_out_sample,
            'max_drawdown_dollars': max_drawdown_dollars,
            'total_trades': total_trades,
            'in_sample_trades': in_sample_trades,
            'out_sample_trades': out_sample_trades,
            'in_sample_pnl': in_sample_pnl,
            'out_sample_pnl': out_sample_pnl,
            'avg_position_size': avg_position_size,
            'max_position_size': max_position_size
        }

        return metrics

## 4. Data Gathering Implementation (data_gather.py)
Paste the contents of data_gather.py here

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import json
import time

# Import the read_ts module for data loading
from read_ts import read_ts_ohlcv_dat, read_ts_pnl_dat_multicore

# Import configuration
from input import *
from SMA_Strategy import SMAStrategy


def data_gather_main():
    # Start overall execution timer
    overall_start_time = time.time()
    
    # Setup paths using relative directories
    WORKING_DIR = "."  # Current directory
    DATA_DIR = os.path.join(WORKING_DIR, "data")

    # Define SYMBOL based on TICKER
    SYMBOL = TICKER.replace('=F', '')  # This strips the '=F' part if present in the ticker symbol

    # Create the output directory for each symbol
    output_dir = os.path.join(WORKING_DIR, 'output', SYMBOL)  # Symbol-specific folder
    os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

    # Function to save plots in the created folder
    def save_plot(plot_name):
        plt.savefig(os.path.join(output_dir, plot_name))  # Save plot to the symbol-specific folder
        plt.show()  # Display the plot
        plt.close()  # Close the plot to free up memory

    # Function to save parameters to a JSON file
    def save_parameters():
        """Save the big_point_value and dynamic_slippage to a JSON file."""
        parameters = {
            "big_point_value": big_point_value,
            "slippage": slippage,
            "capital": TRADING_CAPITAL,
            "atr_period": ATR_PERIOD
        }

        with open("parameters.json", "w") as file:
            json.dump(parameters, file)

    # Function to get slippage value from an Excel file - STRICT VERSION
    def get_slippage_from_excel(symbol, data_dir):
        """
        Get the slippage value for a specific symbol from the Excel file
        
        Parameters:
        symbol: str - The trading symbol to look up (without '=F')
        data_dir: str - Directory containing the Excel file
        
        Returns:
        float - Slippage value for the symbol
        """
        excel_path = os.path.join(data_dir, "sessions_slippages.xlsx")
        
        # No fallback - if file doesn't exist, crash
        if not os.path.exists(excel_path):
            raise FileNotFoundError(f"Slippage Excel file not found at {excel_path}")
        
        # Remove '=F' suffix if present for lookup
        lookup_symbol = symbol.replace('=F', '')
        
        # Read the Excel file - will throw exception if any issue
        df = pd.read_excel(excel_path)
        
        # Print the Excel contents for debugging
        print("\nContents of sessions_slippages.xlsx:")
        print(df.head())
        
        # Check if we have at least 4 columns (to access column D)
        if df.shape[1] < 4:
            raise ValueError(f"Excel file has fewer than 4 columns: {df.columns.tolist()}")
            
        # Print column names for debugging
        print(f"Columns: {df.columns.tolist()}")
        
        # Use direct column access - Column B (index 1) for symbol, Column D (index 3) for slippage
        # First convert to uppercase for case-insensitive comparison
        df['SymbolUpper'] = df.iloc[:, 1].astype(str).str.upper()
        lookup_symbol_upper = lookup_symbol.upper()
        
        # Find the matching row
        matching_rows = df[df['SymbolUpper'] == lookup_symbol_upper]
        
        if matching_rows.empty:
            raise ValueError(f"Symbol '{lookup_symbol}' not found in column B of Excel file")
            
        # Get the slippage value from column D (index 3)
        slippage_value = matching_rows.iloc[0, 3]
        
        # Validate the slippage value is numeric
        if pd.isna(slippage_value) or not isinstance(slippage_value, (int, float)):
            raise ValueError(f"Invalid slippage value for symbol '{lookup_symbol}': {slippage_value}")
            
        print(f"Found slippage for {lookup_symbol} in column D: {slippage_value}")
        return slippage_value

    # Function to find the data file for the specified futures symbol
    def find_futures_file(symbol, data_dir):
        """Find a data file for the specified futures symbol"""
        # First try a pattern that specifically looks for @SYMBOL
        pattern = f"*@{symbol}_*.dat"
        files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Try a pattern that looks for the symbol with an underscore or at boundary
            pattern = f"*_@{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Try a more specific boundary pattern for the symbol
            pattern = f"*_{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Last resort: less specific but better than nothing
            pattern = f"*@{symbol}*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        # No fallback - if no file found, crash
        if not files:
            raise FileNotFoundError(f"No data file found for {symbol} in {data_dir}")
            
        return files[0]  # Return the first matching file

    # Get symbol from the TICKER variable (remove '=F' if it exists)
    SYMBOL = TICKER.replace('=F', '')

    # Load the futures data file using multicore processing for better performance
    print("Loading data file...")
    load_start_time = time.time()
    
    # First find all matching files
    data_files = glob.glob(os.path.join(DATA_DIR, f"*@{SYMBOL}*.dat"))
    if not data_files:
        data_files = glob.glob(os.path.join(DATA_DIR, f"*_{SYMBOL}_*.dat"))
    
    if not data_files:
        raise FileNotFoundError(f"No data file found for {SYMBOL} in {DATA_DIR}")
    
    # Load data using multicore processing
    all_data = read_ts_ohlcv_dat(data_files[0])  # Use first matching file
    load_end_time = time.time()
    load_time = load_end_time - load_start_time
    print(f"Data loaded successfully in {load_time:.2f} seconds! Number of items: {len(all_data)}")
    
    # Extract metadata and OHLCV data from the first data object
    data_obj = all_data[0]
    tick_size = data_obj.big_point_value * data_obj.tick_size
    
    # Get big point value from data
    big_point_value = data_obj.big_point_value
    
    # Get OHLCV data
    ohlc_data = data_obj.data.copy()  # Make a copy to avoid modifying original data

    # Fetch slippage value from Excel - NO FALLBACK
    slippage_value = get_slippage_from_excel(TICKER, DATA_DIR)
    slippage = slippage_value
    print(f"Using slippage from Excel column D: {slippage}")

    # Save the parameters to a JSON file
    save_parameters()
    
    # Start timing data preparation
    prep_start_time = time.time()
    
    # Print information about the data
    print(f"\nSymbol: {data_obj.symbol}")
    print(f"Description: {data_obj.description}")
    print(f"Exchange: {data_obj.exchange}")
    print(f"Interval: {data_obj.interval_type} {data_obj.interval_span}")
    print(f"Tick size: {tick_size}")
    print(f"Big point value: {big_point_value}")
    print(f"Data shape: {ohlc_data.shape}")
    print(f"Date range: {ohlc_data['datetime'].min()} to {ohlc_data['datetime'].max()}")
    
    # Display the first few rows of data
    print("\nFirst few rows of OHLCV data:")
    print(ohlc_data.head())
    
    # Convert the OHLCV data to the format expected by the SMA strategy
    # First, rename columns to match what yfinance provides
    data = ohlc_data.rename(columns={
        'datetime': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    
    # Set the datetime column as the index
    data.set_index('Date', inplace=True)
    
    # Add warm-up period for SMA calculation
    original_start_idx = None
    
    # Filter data to match the date range if specified in input.py
    if START_DATE and END_DATE:
        # Calculate warm-up period (longest SMA + buffer for ATR calculation)
        warm_up_days = SMA_MAX + ATR_PERIOD + 50  # Add buffer days for safety
        
        # Convert dates to datetime
        start_date = pd.to_datetime(START_DATE)
        end_date = pd.to_datetime(END_DATE)
        
        # Adjust start date for warm-up
        adjusted_start = start_date - pd.Timedelta(days=warm_up_days)
        
        # Load more data for warm-up
        data = data[(data.index >= adjusted_start) & 
                    (data.index <= end_date)]
        
        # Store the original start date index for later use
        if data.empty:
            raise ValueError(f"No data available for the specified date range: {START_DATE} to {END_DATE}")
            
        # Find the closest index to our original start date
        original_start_idx = data.index.get_indexer([start_date], method='nearest')[0]
        
        print(f"Loaded extended data with {warm_up_days} days warm-up period")
        print(f"Original date range: {START_DATE} to {END_DATE}")
        print(f"Adjusted date range: {adjusted_start.strftime('%Y-%m-%d')} to {END_DATE}")
        print(f"Original start index: {original_start_idx}")
    
    prep_end_time = time.time()
    prep_time = prep_end_time - prep_start_time
    print(f"Data preparation completed in {prep_time:.2f} seconds")
    
    # Define the range of SMA periods to test
    sma_range = range(SMA_MIN, SMA_MAX + 1, SMA_STEP)
    
    print(f"Optimizing SMA parameters using range from {SMA_MIN} to {SMA_MAX} with step {SMA_STEP}...")
    print(f"Trading with big point value from data: {big_point_value}")
    print(f"Using capital allocation: ${TRADING_CAPITAL:,} with ATR period: {ATR_PERIOD}")
    
    # Initialize the ATR-based strategy using the big point value from the data
    strategy = SMAStrategy(
        short_sma=0,  # Will be set during optimization
        long_sma=0,  # Will be set during optimization
        big_point_value=big_point_value,  # Use the big point value from data
        slippage=slippage,  # Use dynamically calculated slippage
        capital=TRADING_CAPITAL,  # Capital allocation for position sizing
        atr_period=ATR_PERIOD  # ATR period for position sizing
    )
    
    # Start timing the optimization process
    print("\nStarting optimization process...")
    optimization_start_time = time.time()
    
    # Run the optimization function to find the best SMA parameters
    # Save results file for data_analysis.py to use
    best_sma, best_sharpe, best_trades, all_results = strategy.optimize(
        data.copy(),
        sma_range,
        train_test_split=TRAIN_TEST_SPLIT,
        results_file='sma_all_results.txt',  # Relative path to current directory
        warm_up_idx=original_start_idx  # Pass the warm-up index to ensure consistent Sharpe calculation
    )
    
    # Calculate and display the time taken for optimization
    optimization_end_time = time.time()
    optimization_time = optimization_end_time - optimization_start_time
    print(f"\nOptimization completed in {optimization_time:.2f} seconds ({optimization_time/60:.2f} minutes)")
    
    print(f"Optimal SMA parameters: Short = {best_sma[0]} days, Long = {best_sma[1]} days")
    print(f"In-sample Sharpe ratio = {best_sharpe:.4f}")
    print(f"Number of trades with optimal parameters = {best_trades}")
    print(f"Optimization results saved to 'sma_all_results.txt' for further analysis")
    
    # Update strategy with the best parameters
    strategy.short_sma = best_sma[0]
    strategy.long_sma = best_sma[1]
    
    # Apply the best SMA parameters found from optimization to the dataset
    print("\nApplying best strategy parameters...")
    apply_start_time = time.time()
    
    # Apply strategy to the full dataset including warm-up period
    data = strategy.apply_strategy(data.copy())
    
    # Store the data for evaluation (will be trimmed later)
    data_for_evaluation = data.copy()
    
    apply_end_time = time.time()
    apply_time = apply_end_time - apply_start_time
    print(f"Strategy application completed in {apply_time:.2f} seconds")
    
    # Start timing the visualization process
    viz_start_time = time.time()
    
    # Trim data back to the original date range for evaluation
    if original_start_idx is not None:
        print("Trimming warm-up period for final evaluation and visualization...")
        data_for_evaluation = data.iloc[original_start_idx:]
        print(f"Original data length: {len(data)}, Evaluation data length: {len(data_for_evaluation)}")
    else:
        data_for_evaluation = data
    
    # Create a figure with multiple subplots
    plt.figure(figsize=(14, 16))
    
    # Plot price and SMAs
    plt.subplot(3, 1, 1)
    plt.plot(data_for_evaluation.index, data_for_evaluation['Close'], label=f'{data_obj.symbol} Price', color='blue')
    plt.plot(data_for_evaluation.index, data_for_evaluation['SMA_Short_Strategy'], label=f'{best_sma[0]}-day SMA', color='orange')
    plt.plot(data_for_evaluation.index, data_for_evaluation['SMA_Long_Strategy'], label=f'{best_sma[1]}-day SMA', color='red')
    
    # Plot position changes (using vectorized identification of changes)
    long_entries = (data_for_evaluation['Position_Dir_Strategy'] == 1) & data_for_evaluation['Position_Change_Strategy']
    short_entries = (data_for_evaluation['Position_Dir_Strategy'] == -1) & data_for_evaluation['Position_Change_Strategy']
    
    # Plot the entries
    plt.scatter(data_for_evaluation.index[long_entries], data_for_evaluation.loc[long_entries, 'Close'], 
                color='green', marker='^', s=50, label='Long Entry')
    plt.scatter(data_for_evaluation.index[short_entries], data_for_evaluation.loc[short_entries, 'Close'], 
                color='red', marker='v', s=50, label='Short Entry')
    
    plt.legend()
    plt.title(f'{data_obj.symbol} with Optimized SMA Strategy ({best_sma[0]}, {best_sma[1]})')
    plt.grid(True)
    
    # Plot position size based on ATR with dual y-axes
    ax1 = plt.subplot(3, 1, 2)
    ax2 = ax1.twinx()  # Create a second y-axis that shares the same x-axis
    
    # Plot position size on the left y-axis
    ax1.plot(data_for_evaluation.index, data_for_evaluation['Position_Size_Strategy'], 
             label='Position Size (# Contracts)', color='purple')
    ax1.set_ylabel('Position Size (# Contracts)', color='purple')
    ax1.tick_params(axis='y', colors='purple')
    
    # Plot ATR on the right y-axis
    ax2.plot(data_for_evaluation.index, data_for_evaluation['ATR_Strategy'], 
             label=f'ATR ({ATR_PERIOD}-day)', color='orange')
    ax2.set_ylabel(f'ATR ({ATR_PERIOD}-day)', color='orange')
    ax2.tick_params(axis='y', colors='orange')
    
    # Add legends for both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.title(f'Position Sizing Based on {ATR_PERIOD}-day ATR')
    ax1.grid(True)
    
    # Plot the performance (P&L)
    plt.subplot(3, 1, 3)
    
    # Reset P&L to zero at start of evaluation period for cleaner visualization
    strategy_pnl_cumulative = data_for_evaluation['Cumulative_PnL_Strategy'] - data_for_evaluation['Cumulative_PnL_Strategy'].iloc[0]
    
    # Plot the cumulative P&L of the strategy (removed market P&L)
    plt.plot(data_for_evaluation.index, strategy_pnl_cumulative, 
             label='Strategy P&L (full period)', color='green')
    
    # Calculate split index based on trimmed data
    split_index = int(len(data_for_evaluation) * TRAIN_TEST_SPLIT)
    
    # Highlight out-of-sample period
    plt.plot(data_for_evaluation.index[split_index:], strategy_pnl_cumulative.iloc[split_index:],
            label=f'Strategy P&L (last {int((1 - TRAIN_TEST_SPLIT) * 100)}% out-of-sample)', color='purple')
    
    # Add split line and zero line
    plt.axvline(x=data_for_evaluation.index[split_index], color='black', linestyle='--',
                label=f'Train/Test Split ({int(TRAIN_TEST_SPLIT * 100)}%/{int((1 - TRAIN_TEST_SPLIT) * 100)}%)')
    plt.axhline(y=0.0, color='gray', linestyle='-', label='Break-even')
    
    plt.legend()
    plt.title('Strategy Performance (Dollar P&L)')
    plt.ylabel('P&L ($)')
    plt.grid(True)
    
    plt.tight_layout()
    save_plot('Optimized_Strategy_Plot.png')
    
    viz_end_time = time.time()
    viz_time = viz_end_time - viz_start_time
    print(f"Visualization completed in {viz_time:.2f} seconds")
    
    # Calculate performance metrics
    metrics_start_time = time.time()
    metrics = strategy.calculate_performance_metrics(
        data_for_evaluation,  # Use the trimmed data for metrics
        strategy_name="Strategy",
        train_test_split=TRAIN_TEST_SPLIT
    )
    metrics_end_time = time.time()
    metrics_time = metrics_end_time - metrics_start_time
    print(f"Performance metrics calculation completed in {metrics_time:.2f} seconds")
    
    # Calculate market performance for comparison (for reporting only, not plotting)
    market_cumulative_pnl = data_for_evaluation['Market_PnL_Strategy'].cumsum().iloc[-1]
    
    # Print summary statistics
    print("\n--- PERFORMANCE SUMMARY OF ATR-BASED SMA STRATEGY ---")
    print(f"Symbol: {data_obj.symbol}")
    print(f"Big Point Value (from data): {big_point_value}")
    print(f"ATR Period for Position Sizing: {ATR_PERIOD} days")
    print(f"Capital Allocation: ${TRADING_CAPITAL:,}")
    print(f"Average Position Size: {metrics['avg_position_size']:.2f} contracts")
    print(f"Maximum Position Size: {metrics['max_position_size']:.0f} contracts")
    print(f"Strategy Total P&L: ${metrics['total_pnl']:,.2f}")
    print(f"Market Buy & Hold P&L: ${market_cumulative_pnl:,.2f}")
    print(f"Outperformance: ${(metrics['total_pnl'] - market_cumulative_pnl):,.2f}")
    
    # *** SHARPE RATIO VERIFICATION ***
    print("\n--- SHARPE RATIO COMPARISON VERIFICATION ---")
    print(f"Optimization in-sample Sharpe ratio: {best_sharpe:.6f}")
    print(f"Final in-sample Sharpe ratio: {metrics['sharpe_in_sample']:.6f}")
    print(f"Difference: {abs(best_sharpe - metrics['sharpe_in_sample']):.6f}")
    if abs(best_sharpe - metrics['sharpe_in_sample']) < 0.001:
        print("✓ SHARPE RATIOS MATCH (within 0.001 tolerance)")
    else:
        print("✗ SHARPE RATIOS DO NOT MATCH")
    
    print(f"Sharpe ratio (entire period, annualized): {metrics['sharpe_full']:.4f}")
    print(f"Sharpe ratio (in-sample, annualized): {metrics['sharpe_in_sample']:.4f}")
    print(f"Sharpe ratio (out-of-sample, annualized): {metrics['sharpe_out_sample']:.4f}")
    print(f"Maximum Drawdown: ${abs(metrics['max_drawdown_dollars']):,.2f}")
    print("\n--- TRADE COUNT SUMMARY ---")
    print(f"In-sample period trades: {metrics['in_sample_trades']}")
    print(f"Out-of-sample period trades: {metrics['out_sample_trades']}")
    print(f"Total trades: {metrics['total_trades']}")
    print(f"In-sample P&L: ${metrics['in_sample_pnl']:,.2f}")
    print(f"Out-of-sample P&L: ${metrics['out_sample_pnl']:,.2f}")
    
    print(f"\nBest parameters: Short SMA = {best_sma[0]}, Long SMA = {best_sma[1]}, Sharpe = {best_sharpe:.6f}, Trades = {best_trades}")
    
    # Calculate overall execution time
    overall_end_time = time.time()
    overall_time = overall_end_time - overall_start_time
    
    # Print timing summary
    print("\n--- EXECUTION TIME SUMMARY (Vectorized Implementation) ---")
    print(f"Data loading time: {load_time:.2f} seconds")
    print(f"Data preparation time: {prep_time:.2f} seconds")
    print(f"Optimization time: {optimization_time:.2f} seconds ({optimization_time/60:.2f} minutes)")
    print(f"Strategy application time: {apply_time:.2f} seconds")
    print(f"Visualization time: {viz_time:.2f} seconds")
    print(f"Metrics calculation time: {metrics_time:.2f} seconds")
    print(f"Total execution time: {overall_time:.2f} seconds ({overall_time/60:.2f} minutes)")
    
    print("\nAnalysis complete!")


if __name__ == "__main__":
    data_gather_main()

## 5. Data Analysis Implementation (data_analysis.py)
Paste the contents of data_analysis.py here

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
from matplotlib.patches import Circle
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import calendar
import read_ts
import openpyxl

import os
import matplotlib.pyplot as plt
import matplotlib

from input import MIN_TRADES, MAX_TRADES, MIN_ELEMENTS_PER_CLUSTER, DEFAULT_NUM_CLUSTERS
from input import TICKER, START_DATE, END_DATE, TRAIN_TEST_SPLIT, ATR_PERIOD, TRADING_CAPITAL
from SMA_Strategy import SMAStrategy

import glob
import json
def data_analysis_main():
    def load_parameters():
        try:
            with open("parameters.json", "r") as file:
                parameters = json.load(file)
                big_point_value = parameters["big_point_value"]
                slippage = parameters["slippage"]
                capital = parameters.get("capital", TRADING_CAPITAL)
                atr_period = parameters.get("atr_period", ATR_PERIOD)
                return big_point_value, slippage, capital, atr_period
        except FileNotFoundError:
            print("Parameters file not found. Ensuring it was saved correctly in data_gather.py.")
            return None, None, TRADING_CAPITAL, ATR_PERIOD

    # Load the parameters
    big_point_value, slippage, capital, atr_period = load_parameters()

    print(f"Big Point Value: {big_point_value}")
    print(f"Dynamic Slippage: {slippage}")
    print(f"Capital for Position Sizing: {capital:,}")
    print(f"ATR Period: {atr_period}")

    # Setup paths
    WORKING_DIR = "."  # Current directory
    DATA_DIR = os.path.join(WORKING_DIR, "data")
    SYMBOL = TICKER.replace('=F', '')

    # Define the output folder where the plots will be saved
    output_dir = os.path.join(WORKING_DIR, 'output', SYMBOL)
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Function to save plots in the created folder
    def save_plot(plot_name):
        plt.savefig(os.path.join(output_dir, plot_name))  # Save plot to the symbol-specific folder
        plt.show()  # Display the plot
        plt.close()  # Close the plot to free up memory

    # Function to save results to csv in the same output directory
    def save_results(data_frame, file_name):
        csv_path = os.path.join(output_dir, file_name)
        data_frame.to_csv(csv_path, index=False)
        print(f"Results saved to {csv_path}")

    def find_futures_file(symbol, data_dir):
        """Find a data file for the specified futures symbol"""
        # First try a pattern that specifically looks for @SYMBOL
        pattern = f"*@{symbol}_*.dat"
        files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Try a pattern that looks for the symbol with an underscore or at boundary
            pattern = f"*_@{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Try a more specific boundary pattern for the symbol
            pattern = f"*_{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Last resort: less specific but better than nothing
            pattern = f"*@{symbol}*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        return files[0] if files else None

    # Added functions from cluster_visualization.py
    def compute_medoids(X, labels, valid_clusters):
        """Compute medoids for each cluster (point with minimum distance to all other points in cluster)"""
        medoids = []

        for cluster_id in valid_clusters:
            # Get points in this cluster
            cluster_points = X[labels == cluster_id]

            if len(cluster_points) == 0:
                continue

            # Calculate pairwise distances within cluster
            min_total_distance = float('inf')
            medoid = None

            for i, point1 in enumerate(cluster_points):
                total_distance = 0
                for j, point2 in enumerate(cluster_points):
                    # Calculate Euclidean distance between points
                    distance = np.sqrt(np.sum((point1 - point2) ** 2))
                    total_distance += distance

                if total_distance < min_total_distance:
                    min_total_distance = total_distance
                    medoid = point1

            if medoid is not None:
                medoids.append(medoid)

        return medoids

    def cluster_analysis(file_path='sma_all_results.txt', min_trades=MIN_TRADES, max_trades=MAX_TRADES,
                min_elements_per_cluster=MIN_ELEMENTS_PER_CLUSTER):
        """
        Perform clustering analysis on SMA optimization results to find robust parameter regions
        Now clusters based on short_SMA, long_SMA, and sharpe_ratio only (not trades)
        """
        print(f"\n----- CLUSTER ANALYSIS -----")
        print(f"Loading data from {file_path}...")

        # Load the data
        df = pd.read_csv(file_path)

        # Convert data to numpy array for easier processing
        X_full = df[['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

        # Filter data by number of trades and ensure short_SMA < long_SMA
        X_filtered_full = X_full[(X_full[:, 0] < X_full[:, 1]) &  # short_SMA < long_SMA
                    (X_full[:, 3] >= min_trades) &  # trades >= min_trades
                    (X_full[:, 3] <= max_trades)]  # trades <= max_trades

        if len(X_filtered_full) == 0:
            print(
                f"No data points meet the criteria after filtering! Adjust min_trades ({min_trades}) and max_trades ({max_trades}).")
            return None, None, None, None, None

        # Create a version with only the 3 dimensions for clustering
        X_filtered = X_filtered_full[:, 0:3]  # Only short_SMA, long_SMA, and sharpe_ratio

        print(f"Filtered data to {len(X_filtered)} points with {min_trades}-{max_trades} trades")

        # Extract the fields for better scaling visibility
        short_sma_values = X_filtered[:, 0]
        long_sma_values = X_filtered[:, 1]
        sharpe_values = X_filtered[:, 2]
        trades_values = X_filtered_full[:, 3]

        print(f"Short SMA range: {short_sma_values.min()} to {short_sma_values.max()}")
        print(f"Long SMA range: {long_sma_values.min()} to {long_sma_values.max()}")
        print(f"Sharpe ratio range: {sharpe_values.min():.4f} to {sharpe_values.max():.4f}")
        print(f"Trades range: {trades_values.min()} to {trades_values.max()}")

        # Scale the data for clustering - using StandardScaler for each dimension
        # This addresses the issue where SMA values have much larger ranges than Sharpe ratio
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_filtered)  # Only scale the 3 dimensions we use for clustering

        # Print scaling info for verification
        print("\nScaled data information:")
        scaled_short = X_scaled[:, 0]
        scaled_long = X_scaled[:, 1]
        scaled_sharpe = X_scaled[:, 2]

        print(f"Scaled Short SMA range: {scaled_short.min():.4f} to {scaled_short.max():.4f}")
        print(f"Scaled Long SMA range: {scaled_long.min():.4f} to {scaled_long.max():.4f}")
        print(f"Scaled Sharpe ratio range: {scaled_sharpe.min():.4f} to {scaled_sharpe.max():.4f}")

        # Determine number of clusters
        print(f"Using default number of clusters: {DEFAULT_NUM_CLUSTERS}")
        k = DEFAULT_NUM_CLUSTERS

        # Apply KMeans clustering
        print(f"Performing KMeans clustering with k={k}...")
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)

        # Get cluster labels
        labels = kmeans.labels_

        # Count elements per cluster
        unique_labels, counts = np.unique(labels, return_counts=True)
        cluster_sizes = dict(zip(unique_labels, counts))

        print("\nCluster sizes:")
        for cluster_id, size in cluster_sizes.items():
            print(f"Cluster {cluster_id}: {size} elements")

        # Filter clusters with enough elements
        valid_clusters = {i for i, count in cluster_sizes.items() if count >= min_elements_per_cluster}
        filtered_indices = np.array([i in valid_clusters for i in labels])

        # Filter data to only include points in valid clusters
        X_valid = X_filtered_full[filtered_indices]  # Use full data to get trades too
        labels_valid = labels[filtered_indices]

        # Compute medoids using the existing method that expects 4D data
        print("Computing medoids...")
        medoids = compute_medoids(X_valid, labels_valid, valid_clusters)

        # Compute centroids (only for the 3 dimensions we clustered on)
        centroids_scaled = kmeans.cluster_centers_
        # Create a version that can be inverse-transformed (matches the original dimension count)
        centroids = np.zeros((centroids_scaled.shape[0], 3))
        centroids[:, 0:3] = scaler.inverse_transform(centroids_scaled)

        # Print raw centroids for debugging
        print("\nCluster Centroids (in original space):")
        for i, centroid in enumerate(centroids):
            if i in valid_clusters:  # Only show valid clusters
                print(f"Centroid {i}: Short SMA={centroid[0]:.2f}, Long SMA={centroid[1]:.2f}, "
                    f"Sharpe={centroid[2]:.4f}")

        # Simply take top 5 medoids by Sharpe ratio
        top_medoids = sorted(medoids, key=lambda x: float(x[2]), reverse=True)[:5]
        
        # Debug print the sorted top medoids
        print("\nSELECTED TOP 5 MEDOIDS BY SHARPE RATIO:")
        for idx, medoid in enumerate(top_medoids, 1):
            print(f"Top {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

        # Find max Sharpe ratio point overall
        max_sharpe_idx = np.argmax(df['sharpe_ratio'].values)
        max_sharpe_point = df.iloc[max_sharpe_idx][['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

        # Print results
        print("\n----- CLUSTERING RESULTS -----")
        print(f"Max Sharpe point: Short SMA={int(max_sharpe_point[0])}, Long SMA={int(max_sharpe_point[1])}, "
            f"Sharpe={max_sharpe_point[2]:.4f}, Trades={int(max_sharpe_point[3])}")

        print("\nTop 5 Medoids (by Sharpe ratio):")
        for idx, medoid in enumerate(top_medoids, 1):
            print(f"Top {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

        # Create visualization with clustering results - pass the original labels and valid_clusters
        create_cluster_visualization(X_filtered_full, medoids, top_medoids, centroids, max_sharpe_point, 
                                labels=labels, valid_clusters=valid_clusters)

        return X_filtered_full, medoids, top_medoids, centroids, max_sharpe_point

    def create_cluster_visualization(X_filtered_full, medoids, top_medoids, centroids, max_sharpe_point, labels=None, valid_clusters=None):
        """
        Create a continuous heatmap visualization with cluster centers overlaid.
        Only plots data points and clusters that meet the filtering criteria.
        
        Parameters:
        X_filtered_full: array - Filter-compliant data points with shape (n_samples, 4) containing short_SMA, long_SMA, sharpe_ratio, trades
        medoids: list - List of medoids from valid clusters
        top_medoids: list - List of top medoids by Sharpe ratio
        centroids: array - Centroids of clusters
        max_sharpe_point: array - Point with maximum Sharpe ratio
        labels: array - Cluster labels for each point in X_filtered_full (optional)
        valid_clusters: set - Set of valid cluster IDs that meet the min_elements_per_cluster requirement (optional)
        """
        print("Creating cluster visualization...")

        # Load the full dataset, but we'll only use it for creating the heatmap grid
        data = pd.read_csv('sma_all_results.txt')
        
        # Create filtered dataframe from the filtered points that meet trade requirements
        # This ensures we only visualize points that meet the trade count requirements
        filtered_df = pd.DataFrame(X_filtered_full, columns=['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades'])
        
        # Create a pivot table for the heatmap using ONLY the filtered data points
        heatmap_data = filtered_df.pivot_table(
            index='long_SMA',
            columns='short_SMA',
            values='sharpe_ratio',
            fill_value=np.nan  # Use NaN for empty cells
        )

        # Create the heatmap visualization
        plt.figure(figsize=(12, 10))

        # Create mask for invalid combinations (where short_SMA >= long_SMA)
        # and also for NaN values (which represent filtered-out points)
        mask = np.zeros_like(heatmap_data, dtype=bool)
        for i, long_sma in enumerate(heatmap_data.index):
            for j, short_sma in enumerate(heatmap_data.columns):
                if short_sma >= long_sma or np.isnan(heatmap_data.iloc[i, j]):
                    mask[i, j] = True

        # Plot the base heatmap with ONLY filtered data
        ax = sns.heatmap(
            heatmap_data,
            mask=mask,
            cmap='coolwarm',  # Blue to red colormap
            annot=False,      # Don't annotate each cell with its value
            fmt='.4f',
            linewidths=0,
            cbar_kws={'label': 'Sharpe Ratio'}
        )

        # Invert the y-axis so smaller long_SMA values are at the top
        ax.invert_yaxis()

        # Plot max Sharpe point (Green Star)
        # Note: We need to convert from data values to plot coordinates
        try:
            best_x_pos = np.where(heatmap_data.columns == max_sharpe_point[0])[0][0] + 0.5
            best_y_pos = np.where(heatmap_data.index == max_sharpe_point[1])[0][0] + 0.5
            plt.scatter(best_x_pos, best_y_pos, marker='*', color='lime', s=200,
                        edgecolor='black', zorder=5)
        except IndexError:
            print(f"Warning: Max Sharpe point at ({max_sharpe_point[0]}, {max_sharpe_point[1]}) not found in heatmap coordinates")

        # Only plot medoids from valid clusters
        if medoids:
            for medoid in medoids:
                try:
                    x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
                    y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
                    plt.scatter(x_pos, y_pos, marker='s', color='black', s=75, zorder=4)
                except IndexError:
                    print(f"Warning: Medoid at ({medoid[0]}, {medoid[1]}) not found in heatmap coordinates")

        # Plot top 5 medoids (Purple Diamonds)
        if top_medoids:
            for medoid in top_medoids:
                try:
                    x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
                    y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
                    plt.scatter(x_pos, y_pos, marker='D', color='purple', s=100, zorder=5)
                except IndexError:
                    print(f"Warning: Top medoid at ({medoid[0]}, {medoid[1]}) not found in heatmap coordinates")

        # Only plot centroids from valid clusters
        print(f"Plotting centroids from valid clusters...")
        centroids_plotted = 0

        # If valid_clusters is not provided, assume all centroids are valid
        plot_centroids = centroids
        if valid_clusters is not None and labels is not None:
            # Only plot centroids from valid clusters
            plot_centroids = [centroids[i] for i in range(len(centroids)) if i in valid_clusters]
            print(f"Filtering centroids to only include valid clusters: {valid_clusters}")
        
        for i, centroid in enumerate(plot_centroids):
            # Get the actual raw centroid values
            short_sma = centroid[0]
            long_sma = centroid[1]

            print(f"Centroid {i}: raw values = ({short_sma}, {long_sma})")

            # First try exact values
            try:
                if (short_sma in heatmap_data.columns) and (long_sma in heatmap_data.index) and (short_sma < long_sma):
                    x_pos = np.where(heatmap_data.columns == short_sma)[0][0] + 0.5
                    y_pos = np.where(heatmap_data.index == long_sma)[0][0] + 0.5
                    plt.scatter(x_pos, y_pos, marker='o', color='blue', s=75, zorder=4)
                    centroids_plotted += 1
                    continue
            except (IndexError, TypeError):
                pass

            # Try rounded values
            try:
                short_sma_rounded = int(round(short_sma))
                long_sma_rounded = int(round(long_sma))

                print(f"  Rounded: ({short_sma_rounded}, {long_sma_rounded})")

                if (short_sma_rounded in heatmap_data.columns) and (long_sma_rounded in heatmap_data.index) and (
                        short_sma_rounded < long_sma_rounded):
                    x_pos = np.where(heatmap_data.columns == short_sma_rounded)[0][0] + 0.5
                    y_pos = np.where(heatmap_data.index == long_sma_rounded)[0][0] + 0.5
                    plt.scatter(x_pos, y_pos, marker='o', color='blue', s=75, zorder=4)
                    centroids_plotted += 1
                    continue
            except (IndexError, TypeError):
                pass

            # Try finding nearest valid point
            try:
                # Find nearest valid parameter values
                short_options = np.array(heatmap_data.columns)
                long_options = np.array(heatmap_data.index)

                # Find nearest short SMA
                short_idx = np.argmin(np.abs(short_options - short_sma))
                short_nearest = short_options[short_idx]

                # Find nearest long SMA
                long_idx = np.argmin(np.abs(long_options - long_sma))
                long_nearest = long_options[long_idx]

                print(f"  Nearest: ({short_nearest}, {long_nearest})")

                # Check if valid
                if short_nearest < long_nearest:
                    x_pos = np.where(heatmap_data.columns == short_nearest)[0][0] + 0.5
                    y_pos = np.where(heatmap_data.index == long_nearest)[0][0] + 0.5
                    plt.scatter(x_pos, y_pos, marker='o', color='blue', s=75, zorder=4, alpha=0.7)
                    centroids_plotted += 1
                else:
                    print(f"  Invalid nearest parameters (short >= long): {short_nearest} >= {long_nearest}")
            except (IndexError, TypeError) as e:
                print(f"  Error finding nearest point: {e}")

        print(f"Successfully plotted {centroids_plotted} out of {len(plot_centroids)} centroids")

        # Create custom legend
        max_sharpe_handle = mlines.Line2D([], [], color='lime', marker='*', linestyle='None',
                                        markersize=15, markeredgecolor='black', label='Max Sharpe')
        medoid_handle = mlines.Line2D([], [], color='black', marker='s', linestyle='None',
                                    markersize=10, label='Medoids')
        top_medoid_handle = mlines.Line2D([], [], color='purple', marker='D', linestyle='None',
                                        markersize=10, label='Top 5 Medoids')
        centroid_handle = mlines.Line2D([], [], color='blue', marker='o', linestyle='None',
                                        markersize=10, label='Centroids')

        # Add legend
        plt.legend(handles=[max_sharpe_handle, medoid_handle, top_medoid_handle, centroid_handle],
                loc='best')

        # Set labels and title
        plt.title('SMA Parameter Clustering Analysis (Sharpe Ratio)', fontsize=14)
        plt.xlabel('Short SMA (days)', fontsize=12)
        plt.ylabel('Long SMA (days)', fontsize=12)

        # Rotate tick labels
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)

        # Display plot
        plt.tight_layout()
        save_plot('Cluster_Analysis.png')

    # Load the SMA simulation results
    def analyze_sma_results(file_path='sma_all_results.txt'):
        print(f"Loading simulation results from {file_path}...")

        # Load the data from the CSV file
        data = pd.read_csv(file_path)

        # Print basic information about the data
        print(f"Loaded {len(data)} simulation results")
        print(f"Short SMA range: {data['short_SMA'].min()} to {data['short_SMA'].max()}")
        print(f"Long SMA range: {data['long_SMA'].min()} to {data['long_SMA'].max()}")
        print(f"Sharpe ratio range: {data['sharpe_ratio'].min():.4f} to {data['sharpe_ratio'].max():.4f}")

        # Find the best Sharpe ratio
        best_idx = data['sharpe_ratio'].idxmax()
        best_short_sma = data.loc[best_idx, 'short_SMA']
        best_long_sma = data.loc[best_idx, 'long_SMA']
        best_sharpe = data.loc[best_idx, 'sharpe_ratio']
        best_trades = data.loc[best_idx, 'trades']

        print(f"\nBest parameters:")
        print(f"Short SMA: {best_short_sma}")
        print(f"Long SMA: {best_long_sma}")
        print(f"Sharpe Ratio: {best_sharpe:.6f}")
        print(f"Number of Trades: {best_trades}")

        # Create a pivot table for the heatmap
        heatmap_data = data.pivot_table(
            index='long_SMA',
            columns='short_SMA',
            values='sharpe_ratio'
        )

        # Create the heatmap visualization
        plt.figure(figsize=(12, 10))

        # Create a mask for invalid combinations (where short_SMA >= long_SMA)
        mask = np.zeros_like(heatmap_data, dtype=bool)
        for i, long_sma in enumerate(heatmap_data.index):
            for j, short_sma in enumerate(heatmap_data.columns):
                if short_sma >= long_sma:
                    mask[i, j] = True

        # Plot the heatmap with the mask
        ax = sns.heatmap(
            heatmap_data,
            mask=mask,
            cmap='coolwarm',  # Blue to red colormap
            annot=False,  # Don't annotate each cell with its value
            fmt='.4f',
            linewidths=0,
            cbar_kws={'label': 'Sharpe Ratio'}
        )

        # Invert the y-axis so smaller long_SMA values are at the top
        ax.invert_yaxis()

        # Find the position of the best Sharpe ratio in the heatmap
        best_y = heatmap_data.index.get_loc(best_long_sma)
        best_x = heatmap_data.columns.get_loc(best_short_sma)

        # Add a star to mark the best Sharpe ratio
        # We need to add 0.5 to center the marker in the cell
        ax.add_patch(Circle((best_x + 0.5, best_y + 0.5), 0.4, facecolor='none',
                            edgecolor='white', lw=2))
        plt.plot(best_x + 0.5, best_y + 0.5, 'w*', markersize=10)

        # Set labels and title
        plt.title(f'SMA Optimization Heatmap (Best Sharpe: {best_sharpe:.4f} at {best_short_sma}/{best_long_sma})',
                fontsize=14)
        plt.xlabel('Short SMA (days)', fontsize=12)
        plt.ylabel('Long SMA (days)', fontsize=12)

        # Rotate tick labels for better readability
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)

        # Add a text annotation for the best parameters
        plt.annotate(
            f'Best: Short={best_short_sma}, Long={best_long_sma}\nSharpe={best_sharpe:.4f}, Trades={best_trades}',
            xy=(best_x + 0.5, best_y + 0.5),
            xytext=(best_x + 5, best_y + 5),
            arrowprops=dict(arrowstyle="->", color='white'),
            color='white',
            backgroundcolor='black',
            bbox=dict(boxstyle="round,pad=0.3", fc="black", alpha=0.7)
        )

        # Display the plot
        plt.tight_layout()
        save_plot('Heatmap.png')

        # Return the data and best parameters
        return data, best_short_sma, best_long_sma, best_sharpe, best_trades


    def plot_strategy_performance(short_sma, long_sma, top_medoids=None, big_point_value=1, slippage=0, capital=1000000, atr_period=14):
        print(f"\n----- PLOTTING STRATEGY PERFORMANCE -----")
        print(f"Using Short SMA: {short_sma}, Long SMA: {long_sma}")
        print(f"Trading with ATR-based position sizing (Capital: ${capital:,}, ATR Period: {atr_period})")
        if top_medoids:
            print(f"Including top {len(top_medoids)} medoids")

        # Load data from local file
        print(f"Loading {TICKER} data from local files...")
        data_file = find_futures_file(SYMBOL, DATA_DIR)
        if not data_file:
            print(f"Error: No data file found for {TICKER} in {DATA_DIR}")
            exit(1)

        print(f"Found data file: {os.path.basename(data_file)}")
        print(f"File size: {os.path.getsize(data_file)} bytes")

        # Load the data from local file
        all_data = read_ts.read_ts_ohlcv_dat(data_file)
        data_obj = all_data[0]
        ohlc_data = data_obj.data.copy()

        # Convert the OHLCV data to the format expected by the strategy
        data = ohlc_data.rename(columns={
            'datetime': 'Date',
            'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close': 'Close',
            'volume': 'Volume'
        })
        data.set_index('Date', inplace=True)

        # Add a warm-up period before the start date
        original_start_idx = None
        if START_DATE and END_DATE:
            # Calculate warm-up period for SMA calculation (longest SMA + buffer)
            warm_up_days = max(short_sma, long_sma) * 3  # Use 3x the longest SMA as warm-up
            
            # Convert dates to datetime
            start_date = pd.to_datetime(START_DATE)
            end_date = pd.to_datetime(END_DATE)
            
            # Adjust start date for warm-up
            adjusted_start = start_date - pd.Timedelta(days=warm_up_days)
            
            # Filter with the extended date range
            extended_data = data[(data.index >= adjusted_start) & (data.index <= end_date)]
            
            # Store the index where the actual analysis should start
            if not extended_data.empty:
                # Find the closest index to our original start date
                original_start_idx = extended_data.index.get_indexer([start_date], method='nearest')[0]
            
            print(f"Added {warm_up_days} days warm-up period before {START_DATE}")
            data = extended_data
        
        # Create a dictionary to store results for each strategy
        strategies = {
            'Best': {'short_sma': short_sma, 'long_sma': long_sma}
        }

        # Add medoids in their original order, including original Sharpe and Trades
        if top_medoids:
            print("\nUSING THESE MEDOIDS (IN ORIGINAL ORDER):")
            for i, medoid in enumerate(top_medoids, 1):
                strategies[f'Medoid {i}'] = {
                    'short_sma': int(medoid[0]),
                    'long_sma': int(medoid[1]),
                    'original_sharpe': float(medoid[2]),  # Store the original Sharpe ratio
                    'original_trades': int(medoid[3])     # Store the original number of trades
                }
                print(f"Medoid {i}: SMA({int(medoid[0])}/{int(medoid[1])}) - Original Sharpe: {float(medoid[2]):.4f}, Trades: {int(medoid[3])}")

        # Apply the proper strategy for each parameter set
        for name, params in strategies.items():
            # Calculate centered SMAs directly
            data[f'SMA_Short_{name}'] = data['Close'].rolling(window=params['short_sma'], center=True).mean()
            data[f'SMA_Long_{name}'] = data['Close'].rolling(window=params['long_sma'], center=True).mean()
            
            # Create a strategy instance for each parameter set
            sma_strategy = SMAStrategy(
                short_sma=params['short_sma'],
                long_sma=params['long_sma'],
                big_point_value=big_point_value,
                slippage=slippage,
                capital=capital,
                atr_period=atr_period
            )

            # Apply the strategy
            data = sma_strategy.apply_strategy(
                data.copy(),
                strategy_name=name
            )

        # Trim data to the original date range if we added warm-up period
        if original_start_idx is not None:
            data_for_evaluation = data.iloc[original_start_idx:]
            print(f"Trimmed warm-up period. Original data length: {len(data)}, Evaluation data length: {len(data_for_evaluation)}")
        else:
            data_for_evaluation = data
        
        # Calculate split index for in-sample/out-of-sample using the trimmed data
        split_index = int(len(data_for_evaluation) * TRAIN_TEST_SPLIT)
        split_date = data_for_evaluation.index[split_index]

        # Create color palette for strategies
        colors = ['blue', 'green', 'purple', 'orange', 'brown', 'pink']

        # Create the performance visualization with just two panels
        plt.figure(figsize=(14, 12))

        # Plot price and SMA (first subplot)
        plt.subplot(2, 1, 1)
        plt.plot(data_for_evaluation.index, data_for_evaluation['Close'], label=f'{SYMBOL} Price', color='black', alpha=0.5)
        
        # Use the centered SMAs for plotting
        for name, params in strategies.items():
            if name == 'Best':
                plt.plot(data_for_evaluation.index, data_for_evaluation[f'SMA_Short_{name}'], 
                        label=f'Short SMA ({params["short_sma"]})', color='orange')
                plt.plot(data_for_evaluation.index, data_for_evaluation[f'SMA_Long_{name}'], 
                        label=f'Long SMA ({params["long_sma"]})', color='blue')
        
        # Mark the train/test split
        plt.axvline(x=split_date, color='black', linestyle='--', alpha=0.7)
        
        plt.legend(loc='upper left')
        plt.title(f'{SYMBOL} Price and SMA Indicators')
        plt.ylabel('Price')
        plt.grid(True, alpha=0.3)

        # Plot cumulative P&L (second subplot)
        plt.subplot(2, 1, 2)

        for i, (name, params) in enumerate(strategies.items()):
            color = colors[i % len(colors)]

            # Plot full period P&L
            plt.plot(data_for_evaluation.index, data_for_evaluation[f'Cumulative_PnL_{name}'],
                    label=f'{name} ({params["short_sma"]}/{params["long_sma"]})', color=color)

            # Plot out-of-sample portion with thicker line
            plt.plot(data_for_evaluation.index[split_index:], data_for_evaluation[f'Cumulative_PnL_{name}'].iloc[split_index:],
                    color=color, linewidth=2.5, alpha=0.7)

        plt.axvline(x=split_date, color='black', linestyle='--',
                    label=f'Train/Test Split ({int(TRAIN_TEST_SPLIT * 100)}%/{int((1 - TRAIN_TEST_SPLIT) * 100)}%)')
        plt.axhline(y=0.0, color='gray', linestyle='-', alpha=0.5, label='Break-even')
        plt.legend(loc='upper left')
        plt.title('Strategy Cumulative P&L')
        plt.ylabel('P&L ($)')
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        save_plot('Multiple_Strategy_Plots.png')

        # Create a list to store performance data for saving to file
        performance_data = []

        # Print detailed performance metrics for all strategies with in-sample and out-of-sample breakdown
        print("\n----- PERFORMANCE SUMMARY -----")

        # IN-SAMPLE PERFORMANCE
        print("\nIN-SAMPLE PERFORMANCE:")
        header = f"{'Strategy':<10} | {'Short/Long':<10} | {'P&L':>12} | {'Sharpe':>7} | {'Trades':>6}"
        separator = "-" * len(header)
        print(separator)
        print(header)
        print(separator)

        # Calculate in-sample metrics using trimmed data
        for name, params in strategies.items():
            short = params['short_sma']
            long = params['long_sma']

            # Get in-sample data from trimmed dataset
            in_sample = data_for_evaluation.iloc[:split_index]

            # Calculate in-sample metrics
            in_sample_daily_pnl = in_sample[f'Daily_PnL_{name}']
            in_sample_cumulative_pnl = in_sample[f'Daily_PnL_{name}'].sum()
            
            # Calculate average position size for in-sample
            avg_pos_size = in_sample[f'Position_Size_{name}'].mean()

            # Use original Sharpe and Trades for medoids, calculate for Best
            if name.startswith('Medoid'):
                in_sample_sharpe = params['original_sharpe']
                in_sample_trades = params['original_trades']
            else:
                # Calculate Sharpe ratio (annualized) for Best
                if in_sample_daily_pnl.std() > 0:
                    in_sample_sharpe = in_sample_daily_pnl.mean() / in_sample_daily_pnl.std() * np.sqrt(252)
                else:
                    in_sample_sharpe = 0
                # Count in-sample trades for Best
                in_sample_trades = in_sample[f'Position_Change_{name}'].sum()
            
            # Print in the original order
            row = f"{name:<10} | {short:>4}/{long:<5} | ${in_sample_cumulative_pnl:>10,.2f} | {in_sample_sharpe:>6.3f} | {in_sample_trades:>6}"
            print(row)
            
            # Store data for later use
            performance_data.append({
                'Period': 'In-Sample',
                'Strategy': name,
                'Short_SMA': short,
                'Long_SMA': long,
                'PnL': in_sample_cumulative_pnl,
                'Avg_Position': avg_pos_size,
                'Sharpe': in_sample_sharpe,
                'Trades': in_sample_trades
            })

        print(separator)

        # OUT-OF-SAMPLE PERFORMANCE
        print("\nOUT-OF-SAMPLE PERFORMANCE:")
        print(separator)
        print(header)
        print(separator)

        # Calculate out-of-sample metrics using trimmed data
        for name, params in strategies.items():
            short = params['short_sma']
            long = params['long_sma']

            # Get out-of-sample data from trimmed dataset
            out_sample = data_for_evaluation.iloc[split_index:]

            # Calculate out-of-sample metrics
            out_sample_daily_pnl = out_sample[f'Daily_PnL_{name}']
            out_sample_cumulative_pnl = out_sample[f'Daily_PnL_{name}'].sum()
            
            # Calculate average position size for out-of-sample
            avg_pos_size = out_sample[f'Position_Size_{name}'].mean()

            # Calculate Sharpe ratio (annualized)
            if out_sample_daily_pnl.std() > 0:
                out_sample_sharpe = out_sample_daily_pnl.mean() / out_sample_daily_pnl.std() * np.sqrt(252)
            else:
                out_sample_sharpe = 0

            # Count out-of-sample trades
            out_sample_trades = out_sample[f'Position_Change_{name}'].sum()
            
            # Print in the original order
            row = f"{name:<10} | {short:>4}/{long:<5} | ${out_sample_cumulative_pnl:>10,.2f} | {out_sample_sharpe:>6.3f} | {out_sample_trades:>6}"
            print(row)
            
            # Store data for later use
            performance_data.append({
                'Period': 'Out-of-Sample',
                'Strategy': name,
                'Short_SMA': short,
                'Long_SMA': long,
                'PnL': out_sample_cumulative_pnl,
                'Avg_Position': avg_pos_size,
                'Sharpe': out_sample_sharpe,
                'Trades': out_sample_trades
            })

        print(separator)

        # FULL PERIOD PERFORMANCE
        print("\nFULL PERIOD PERFORMANCE:")
        print(separator)
        print(header)
        print(separator)

        # Calculate full period metrics using trimmed data
        for name, params in strategies.items():
            short = params['short_sma']
            long = params['long_sma']

            # Calculate full period metrics using trimmed data
            full_daily_pnl = data_for_evaluation[f'Daily_PnL_{name}']
            full_cumulative_pnl = full_daily_pnl.sum()
            
            # Calculate average position size for full period
            avg_pos_size = data_for_evaluation[f'Position_Size_{name}'].mean()
            max_pos_size = data_for_evaluation[f'Position_Size_{name}'].max()

            # Calculate Sharpe ratio (annualized)
            if full_daily_pnl.std() > 0:
                full_sharpe = full_daily_pnl.mean() / full_daily_pnl.std() * np.sqrt(252)
            else:
                full_sharpe = 0

            # Count full period trades
            full_trades = data_for_evaluation[f'Position_Change_{name}'].sum()
            
            # Print in the original order
            row = f"{name:<10} | {short:>4}/{long:<5} | ${full_cumulative_pnl:>10,.2f} | {full_sharpe:>6.3f} | {full_trades:>6}"
            print(row)
            
            # Store data for later use
            performance_data.append({
                'Period': 'Full',
                'Strategy': name,
                'Short_SMA': short,
                'Long_SMA': long,
                'PnL': full_cumulative_pnl,
                'Avg_Position': avg_pos_size,
                'Max_Position': max_pos_size,
                'Sharpe': full_sharpe,
                'Trades': full_trades
            })
            
            # Additional metrics for the best strategy
            if name == 'Best':
                print(f"\nAdditional metrics for Best strategy:")
                print(f"Maximum position size: {max_pos_size:.2f} contracts")
                print(f"Average ATR value: {data_for_evaluation['ATR_Best'].mean():.4f}")
                
                # Calculate drawdown using trimmed data
                peak = data_for_evaluation[f'Cumulative_PnL_{name}'].cummax()
                drawdown = data_for_evaluation[f'Cumulative_PnL_{name}'] - peak
                max_drawdown = drawdown.min()
                
                print(f"Maximum drawdown: ${max_drawdown:.2f}")
                
                # Calculate win rate using trimmed data
                daily_win_rate = (data_for_evaluation[f'Daily_PnL_{name}'] > 0).mean() * 100
                print(f"Daily win rate: {daily_win_rate:.2f}%")

        print(separator)

        # Save performance data to CSV in the ticker output directory
        performance_df = pd.DataFrame(performance_data)
        save_results(performance_df, f"{SYMBOL}_performance_summary.csv")

        return data_for_evaluation  # Return the trimmed data instead of full data


    def bimonthly_out_of_sample_comparison(data, best_short_sma, best_long_sma, top_medoids, min_sharpe=0.2, 
                                big_point_value=big_point_value, slippage=slippage,
                                capital=capital, atr_period=atr_period):
        """
        Compare bimonthly (2-month) performance between the best Sharpe strategy and a portfolio of top medoids
        using ATR-based position sizing.
        
        Parameters:
        data: DataFrame with market data
        best_short_sma: int - The short SMA period for the best Sharpe strategy
        best_long_sma: int - The long SMA period for the best Sharpe strategy
        top_medoids: list - List of top medoids, each as (short_sma, long_sma, sharpe, trades)
        min_sharpe: float - Minimum Sharpe ratio threshold for medoids to be included
        big_point_value: float - Big point value for the futures contract
        slippage: float - Slippage value in price units
        capital: float - Capital allocation for position sizing
        atr_period: int - Period for ATR calculation
        """
        print(f"\n----- BIMONTHLY OUT-OF-SAMPLE COMPARISON -----")
        print(f"Best Sharpe: ({best_short_sma}/{best_long_sma})")
        print(f"Using ATR-based position sizing (Capital: ${capital:,}, ATR Period: {atr_period})")
        
        # Handle the case where top_medoids is None
        if top_medoids is None:
            print("No medoids provided. Comparison cannot be performed.")
            return None
        
        # The top_medoids are already sorted by Sharpe ratio in cluster_analysis
        # Just print them for verification
        print("\nUSING TOP MEDOIDS (BY SHARPE RATIO):")
        for i, medoid in enumerate(top_medoids, 1):
            print(f"Medoid {i}: ({int(medoid[0])}/{int(medoid[1])}) - Sharpe: {float(medoid[2]):.4f}, Trades: {int(medoid[3])}")
        
        # Take at most 3 medoids and filter by minimum Sharpe
        filtered_medoids = []
        for i, m in enumerate(top_medoids[:3]):
            # Check if we can access the required elements
            try:
                # Extract Sharpe ratio and check if it meets the threshold
                short_sma = m[0]
                long_sma = m[1]
                sharpe = float(m[2])  # Convert to float to handle numpy types
                trades = m[3]
                
                if sharpe >= min_sharpe:
                    filtered_medoids.append(m)
                    print(f"Selected medoid {i+1} with Sharpe {sharpe:.4f}")
            except (IndexError, TypeError) as e:
                print(f"Error processing medoid: {e}")
        
        if not filtered_medoids:
            print(f"No medoids have a Sharpe ratio >= {min_sharpe}. Comparison cannot be performed.")
            return None
        
        print(f"Creating portfolio of {len(filtered_medoids)} medoids with Sharpe ratio >= {min_sharpe}:")
        for i, medoid in enumerate(filtered_medoids, 1):
            print(f"Final Medoid {i}: ({int(medoid[0])}/{int(medoid[1])}) - Sharpe: {float(medoid[2]):.4f}")
        
        # Download historical data if not already provided
        if data is None:
            # Load data from local file
            print(f"Loading {TICKER} data from local files...")
            data_file = find_futures_file(SYMBOL, DATA_DIR)
            if not data_file:
                print(f"Error: No data file found for {TICKER} in {DATA_DIR}")
                exit(1)
            
            print(f"Found data file: {os.path.basename(data_file)}")
            print(f"File size: {os.path.getsize(data_file)} bytes")

            # Load the data from local file
            all_data = read_ts.read_ts_ohlcv_dat(data_file)
            data_obj = all_data[0]
            ohlc_data = data_obj.data.copy()

            # Convert the OHLCV data to the format expected by the strategy
            data = ohlc_data.rename(columns={
                'datetime': 'Date',
                'open': 'Open',
                'high': 'High',
                'low': 'Low',
                'close': 'Close',
                'volume': 'Volume'
            })
            data.set_index('Date', inplace=True)
            
            # Filter data to match the date range if specified in input.py
            if START_DATE and END_DATE:
                data = data[(data.index >= pd.to_datetime(START_DATE)) & (data.index <= pd.to_datetime(END_DATE))]
                print(f"Filtered data to date range: {START_DATE} to {END_DATE}")
        
        # Create strategies
        strategies = {
            'Best': {'short_sma': best_short_sma, 'long_sma': best_long_sma}
        }
        
        # Add filtered medoids
        for i, medoid in enumerate(filtered_medoids, 1):
            strategies[f'Medoid_{i}'] = {'short_sma': int(medoid[0]), 'long_sma': int(medoid[1])}
        
        # Apply each strategy to the data
        for name, params in strategies.items():
            strategy = SMAStrategy(
                short_sma=params['short_sma'],
                long_sma=params['long_sma'],
                big_point_value=big_point_value,
                slippage=slippage,
                capital=capital,
                atr_period=atr_period
            )
            
            # Apply the strategy
            data = strategy.apply_strategy(
                data.copy(),
                strategy_name=name
            )
        
        # Get the out-of-sample split date
        split_index = int(len(data) * TRAIN_TEST_SPLIT)
        split_date = data.index[split_index]
        print(f"Out-of-sample period starts on: {split_date.strftime('%Y-%m-%d')}")
        
        # Get out-of-sample data
        oos_data = data.iloc[split_index:].copy()
        
        # Add a year and bimonthly period columns for grouping (each year has 6 bimonthly periods)
        oos_data['year'] = oos_data.index.year.astype(int)
        oos_data['bimonthly'] = ((oos_data.index.month - 1) // 2 + 1).astype(int)
        
        # Create simplified period labels with just the start month (YYYY-MM)
        oos_data['period_label'] = oos_data.apply(
            lambda row: f"{int(row['year'])}-{int((row['bimonthly'] - 1) * 2 + 1):02d}",
            axis=1
        )
        
        # Create a DataFrame to store bimonthly Sharpe ratios
        bimonthly_sharpe = []
        
        # Group by year and bimonthly period, calculate Sharpe ratio for each period
        for period_label, group in oos_data.groupby('period_label'):
            # Skip periods with too few trading days
            if len(group) < 10:
                continue
                
            # Create a bimonthly result entry
            year, start_month = period_label.split('-')
            year = int(year)
            start_month = int(start_month)
            
            bimonthly_result = {
                'period_label': period_label,
                'date': pd.Timestamp(year=year, month=start_month, day=15),  # Middle of first month in period
                'trading_days': len(group),
            }
            
            # Calculate Sharpe ratio for each strategy in this period
            for name in strategies.keys():
                # Get returns for this strategy in this period
                returns = group[f'Daily_PnL_{name}']
                
                # Calculate Sharpe ratio (annualized)
                if len(returns) > 1 and returns.std() > 0:
                    sharpe = returns.mean() / returns.std() * np.sqrt(252)
                else:
                    sharpe = 0
                    
                bimonthly_result[f'{name}_sharpe'] = sharpe
                bimonthly_result[f'{name}_return'] = returns.sum()  # Total P&L for the period
            
            # Calculate the normalized average of medoid Sharpe ratios
            medoid_sharpes = [bimonthly_result[f'Medoid_{i}_sharpe'] for i in range(1, len(filtered_medoids) + 1)]
            bimonthly_result['Avg_Medoid_sharpe'] = sum(medoid_sharpes) / len(filtered_medoids)
            
            # Calculate the normalized average of medoid returns
            medoid_returns = [bimonthly_result[f'Medoid_{i}_return'] for i in range(1, len(filtered_medoids) + 1)]
            bimonthly_result['Avg_Medoid_return'] = sum(medoid_returns) / len(filtered_medoids)
            
            bimonthly_sharpe.append(bimonthly_result)
        
        # Convert to DataFrame
        bimonthly_sharpe_df = pd.DataFrame(bimonthly_sharpe)
        
        # Sort the DataFrame by date for proper chronological display
        if not bimonthly_sharpe_df.empty:
            bimonthly_sharpe_df = bimonthly_sharpe_df.sort_values('date')
        else:
            print(f"WARNING: No bimonthly periods found for {SYMBOL}. Cannot create chart.")
            return None
        
        # Add rounded values to dataframe for calculations
        bimonthly_sharpe_df['Best_sharpe_rounded'] = np.round(bimonthly_sharpe_df['Best_sharpe'], 2)
        bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] = np.round(bimonthly_sharpe_df['Avg_Medoid_sharpe'], 2)
        
        # Print detailed comparison of Sharpe ratios
        print("\nDetailed Sharpe ratio comparison by period:")
        print(f"{'Period':<12} | {'Best Sharpe':>12} | {'Medoid Portfolio':>16} | {'Difference':>12} | {'Portfolio Wins':<14}")
        print("-" * 80)
        
        for idx, row in bimonthly_sharpe_df.iterrows():
            period = row['period_label']
            best_sharpe = row['Best_sharpe']
            avg_medoid_sharpe = row['Avg_Medoid_sharpe']
            best_rounded = row['Best_sharpe_rounded']
            avg_medoid_rounded = row['Avg_Medoid_sharpe_rounded']
            
            diff = avg_medoid_sharpe - best_sharpe
            portfolio_wins = avg_medoid_sharpe > best_sharpe
            
            print(f"{period:<12} | {best_sharpe:12.6f} | {avg_medoid_sharpe:16.6f} | {diff:12.6f} | {portfolio_wins!s:<14}")
        
        # Calculate win rate using raw values
        portfolio_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe'] > bimonthly_sharpe_df['Best_sharpe'])
        total_periods = len(bimonthly_sharpe_df)
        win_percentage = (portfolio_wins / total_periods) * 100 if total_periods > 0 else 0
        
        # Calculate win rate using rounded values (for alternative comparison)
        rounded_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] > bimonthly_sharpe_df['Best_sharpe_rounded'])
        rounded_win_percentage = (rounded_wins / total_periods) * 100 if total_periods > 0 else 0
        
        print(f"\nBimonthly periods analyzed: {total_periods}")
        print(f"Medoid Portfolio Wins: {portfolio_wins} of {total_periods} periods ({win_percentage:.2f}%)")
        print(f"Using rounded values (2 decimal places): {rounded_wins} of {total_periods} periods ({rounded_win_percentage:.2f}%)")
        
        # Create a bar plot to compare bimonthly Sharpe ratios
        plt.figure(figsize=(14, 8))
        
        # Set up x-axis dates
        x = np.arange(len(bimonthly_sharpe_df))
        width = 0.35  # Width of the bars
        
        # Create bars
        plt.bar(x - width/2, bimonthly_sharpe_df['Best_sharpe'], width, 
            label=f'Best Sharpe ({best_short_sma}/{best_long_sma})', color='blue')
        plt.bar(x + width/2, bimonthly_sharpe_df['Avg_Medoid_sharpe'], width, 
            label=f'Medoid Portfolio ({len(filtered_medoids)} strategies)', color='green')
        
        # Add a horizontal line at Sharpe = 0
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        
        # Create medoid description for the title
        medoid_desc = ", ".join([f"({int(m[0])}/{int(m[1])})" for m in filtered_medoids])

        # Customize the plot - using rounded win percentage instead of raw
        plt.title(f'Bimonthly Sharpe Ratio Comparison (Out-of-Sample Period)\n' + 
                f'Medoid Portfolio [{medoid_desc}] outperformed {rounded_win_percentage:.2f}% of the time', 
                fontsize=14)
        plt.xlabel('Bimonthly Period (Start Month)', fontsize=12)
        plt.ylabel('Sharpe Ratio (Annualized)', fontsize=12)
        
        # Simplified x-tick labels with just the period start month
        plt.xticks(x, bimonthly_sharpe_df['period_label'], rotation=45)
        
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Create legend with both strategies - moved to bottom to avoid overlap with title
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2,
                frameon=True, fancybox=True, framealpha=0.9, fontsize=10)
        
        # Add a text box with rounded win percentage instead of raw - moved to right side
        plt.annotate(f'Medoid Portfolio Win Rate: {rounded_win_percentage:.2f}%\n'
                    f'({rounded_wins} out of {total_periods} periods)\n'
                    f'Portfolio: {len(filtered_medoids)} medoids with Sharpe ≥ {min_sharpe}\n'
                    f'ATR-Based Position Sizing (${capital:,}, {atr_period} days)',
                    xy=(0.7, 0.95), xycoords='axes fraction',
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
                    fontsize=12)
        
        plt.tight_layout(rect=[0, 0, 1, 0.95])  # Add extra space at the bottom for the legend
        save_plot('Bimonthly_Comparison.png')
        
        # Save bimonthly comparison to CSV
        save_results(bimonthly_sharpe_df, f"{SYMBOL}_bimonthly_comparison.csv")
        
        # Save win percentage and medoid parameters to Excel file
        try:
            import openpyxl
            
            # Path to the Excel file
            excel_file = r"C:\Users\Admin\Documents\darbas\Results.xlsx"
            
            # Check if file exists
            if not os.path.exists(excel_file):
                print(f"Excel file not found at: {excel_file}")
                return bimonthly_sharpe_df
                
            print(f"Updating Excel file with K-means results for {SYMBOL}...")
            
            # Load the workbook
            wb = openpyxl.load_workbook(excel_file)
            
            # Get the active sheet
            sheet = wb.active
            
            # Find the row with the ticker symbol or the first empty row
            row = 3  # Start from row 3 (assuming rows 1-2 have headers)
            ticker_row = None
            
            while True:
                cell_value = sheet.cell(row=row, column=1).value
                if cell_value == SYMBOL:
                    # Found the ticker symbol
                    ticker_row = row
                    break
                elif cell_value is None:
                    # Found an empty row
                    ticker_row = row
                    # Write the ticker symbol in column A
                    sheet.cell(row=ticker_row, column=1).value = SYMBOL
                    break
                row += 1
            
            # Round the win percentage to one decimal place
            rounded_win_percentage_1dp = round(rounded_win_percentage, 1)
            
            # Write the win percentage in column B (K-means)
            sheet.cell(row=ticker_row, column=2).value = rounded_win_percentage_1dp
            
            # Write the medoid parameters in the respective cluster columns
            # K-means clusters start at column E (5)
            for i, medoid in enumerate(filtered_medoids):
                if i >= 3:  # Only use up to 3 clusters
                    break
                    
                # Calculate column index: E=5, F=6, G=7 for Cluster1, Cluster2, Cluster3
                column_idx = 5 + i
                
                # Format as "short/long" (e.g., "5/20")
                param_value = f"{int(medoid[0])}/{int(medoid[1])}"
                
                # Write to Excel
                sheet.cell(row=ticker_row, column=column_idx).value = param_value
            
            # Write the best Sharpe parameters in column M (13)
            best_sharpe_params = f"{best_short_sma}/{best_long_sma}"
            sheet.cell(row=ticker_row, column=13).value = best_sharpe_params
            
            # Save the workbook
            wb.save(excel_file)
            
            print(f"Excel file updated successfully. Added {SYMBOL} with K-means win rate {rounded_win_percentage_1dp}% in row {ticker_row}")
            print(f"Added best Sharpe parameters {best_sharpe_params} in column M")
            
        except Exception as e:
            print(f"Error updating Excel file: {e}")
        
        # Return the bimonthly Sharpe ratio data
        return bimonthly_sharpe_df
        # Main execution block
    # Main execution block
    # Set matplotlib backend explicitly
    matplotlib.use('Agg')  # Use non-interactive backend for headless environments

    print("Starting ATR-based SMA strategy analysis...")

    # Run the basic analysis first
    data, best_short, best_long, best_sharpe, best_trades = analyze_sma_results()

    if data is None:
        print("Error: Failed to load or analyze SMA results data.")
        exit(1)

    print("\nProceeding with cluster analysis...")

    # Run the cluster analysis to get medoids
    X_filtered, medoids, top_medoids, centroids, max_sharpe_point = cluster_analysis()

    if X_filtered is None or medoids is None:
        print("Error: Cluster analysis failed.")
        exit(1)

    # Re-sort top_medoids to ensure they're in the right order
    if top_medoids is not None:
        print("Re-sorting top medoids by Sharpe ratio...")
        top_medoids = sorted(top_medoids, key=lambda x: float(x[2]), reverse=True)
        for idx, medoid in enumerate(top_medoids, 1):
            print(f"Verified Medoid {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

    print("\nPlotting strategy performance...")

    # Plot strategy performance with the best parameters AND top medoids using ATR-based position sizing
    market_data = plot_strategy_performance(
        best_short, best_long, top_medoids, 
        big_point_value=big_point_value,
        slippage=slippage,
        capital=capital,
        atr_period=atr_period
    )
    
    # Run the bimonthly out-of-sample comparison between best Sharpe and top medoids
    if top_medoids and len(top_medoids) > 0:
        print("\nPerforming bimonthly out-of-sample comparison...")
        bimonthly_sharpe_df = bimonthly_out_of_sample_comparison(
            market_data, 
            best_short, 
            best_long, 
            top_medoids,  # Pass the entire top_medoids list
            big_point_value=big_point_value,
            slippage=slippage,
            capital=capital,
            atr_period=atr_period
        )
    else:
        print("No top medoids found. Cannot run bimonthly comparison.")
    
    print("\nAnalysis complete! All plots and result files have been saved to the output directory.")
    print(f"Output directory: {output_dir}")

# Call the main function to execute the script
if __name__ == "__main__":
    data_analysis_main()

## 6. Hierarchical Analysis Implementation (data_analysis_hierarchy.py)
Paste the contents of data_analysis_hierarchy.py here

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
from matplotlib.patches import Circle
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform
import calendar
import os
import glob
import read_ts
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from matplotlib.lines import Line2D
import matplotlib
import openpyxl

from input import MIN_TRADES, MAX_TRADES, MIN_ELEMENTS_PER_CLUSTER, DEFAULT_NUM_CLUSTERS
from input import TICKER, START_DATE, END_DATE, TRAIN_TEST_SPLIT, ATR_PERIOD, TRADING_CAPITAL
from SMA_Strategy import SMAStrategy

import json

def data_analysis_hierarchy_main():
    def load_parameters():
        try:
            with open("parameters.json", "r") as file:
                parameters = json.load(file)
                big_point_value = parameters["big_point_value"]
                slippage = parameters["slippage"]
                capital = parameters.get("capital", TRADING_CAPITAL)
                atr_period = parameters.get("atr_period", ATR_PERIOD)
                return big_point_value, slippage, capital, atr_period
        except FileNotFoundError:
            print("Parameters file not found. Ensure it was saved correctly in data_gather.py.")
            return None, None, TRADING_CAPITAL, ATR_PERIOD
        except KeyError as e:
            print(f"Missing key in parameters.json: {e}")
            print(f"Available keys: {', '.join(parameters.keys())}")
            return None, None, TRADING_CAPITAL, ATR_PERIOD

    # Load the parameters
    big_point_value, slippage, capital, atr_period = load_parameters()

    print(f"Big Point Value: {big_point_value}")
    print(f"Dynamic Slippage: {slippage}")
    print(f"Capital for Position Sizing: {capital:,}")
    print(f"ATR Period: {atr_period}")

    # Setup paths
    WORKING_DIR = "."  # Current directory
    DATA_DIR = os.path.join(WORKING_DIR, "data")
    SYMBOL = TICKER.replace('=F', '')

    # Define the output folder where the plots will be saved
    output_dir = os.path.join(WORKING_DIR, 'output', SYMBOL)
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Function to save plots in the created folder
    def save_plot(plot_name):
        plt.savefig(os.path.join(output_dir, plot_name))  # Save plot to the symbol-specific folder
        plt.show()  # Display the plot
        plt.close()  # Close the plot to free up memory

    # Function to save results to csv in the same output directory
    def save_results(data_frame, file_name):
        csv_path = os.path.join(output_dir, file_name)
        data_frame.to_csv(csv_path, index=False)
        print(f"Results saved to {csv_path}")

    def find_futures_file(symbol, data_dir):
        """Find a data file for the specified futures symbol"""
        # First try a pattern that specifically looks for @SYMBOL
        pattern = f"*@{symbol}_*.dat"
        files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Try a pattern that looks for the symbol with an underscore or at boundary
            pattern = f"*_@{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Try a more specific boundary pattern for the symbol
            pattern = f"*_{symbol}_*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        if not files:
            # Last resort: less specific but better than nothing
            pattern = f"*@{symbol}*.dat"
            files = glob.glob(os.path.join(data_dir, pattern))
        
        return files[0] if files else None

    # Load the SMA simulation results
    def analyze_sma_results(file_path='sma_all_results.txt'):
        print(f"Loading simulation results from {file_path}...")

        # Load the data from the CSV file
        data = pd.read_csv(file_path)

        # Print basic information about the data
        print(f"Loaded {len(data)} simulation results")
        print(f"Short SMA range: {data['short_SMA'].min()} to {data['short_SMA'].max()}")
        print(f"Long SMA range: {data['long_SMA'].min()} to {data['long_SMA'].max()}")
        print(f"Sharpe ratio range: {data['sharpe_ratio'].min():.4f} to {data['sharpe_ratio'].max():.4f}")

        # Find the best Sharpe ratio
        best_idx = data['sharpe_ratio'].idxmax()
        best_short_sma = data.loc[best_idx, 'short_SMA']
        best_long_sma = data.loc[best_idx, 'long_SMA']
        best_sharpe = data.loc[best_idx, 'sharpe_ratio']
        best_trades = data.loc[best_idx, 'trades']

        print(f"\nBest parameters:")
        print(f"Short SMA: {best_short_sma}")
        print(f"Long SMA: {best_long_sma}")
        print(f"Sharpe Ratio: {best_sharpe:.6f}")
        print(f"Number of Trades: {best_trades}")

        # Create a pivot table for the heatmap
        heatmap_data = data.pivot_table(
            index='long_SMA',
            columns='short_SMA',
            values='sharpe_ratio'
        )

        # Create the heatmap visualization
        plt.figure(figsize=(12, 10))

        # Create a mask for invalid combinations (where short_SMA >= long_SMA)
        mask = np.zeros_like(heatmap_data, dtype=bool)
        for i, long_sma in enumerate(heatmap_data.index):
            for j, short_sma in enumerate(heatmap_data.columns):
                if short_sma >= long_sma:
                    mask[i, j] = True

        # Plot the heatmap with the mask
        ax = sns.heatmap(
            heatmap_data,
            mask=mask,
            cmap='coolwarm',  # Blue to red colormap
            annot=False,  # Don't annotate each cell with its value
            fmt='.4f',
            linewidths=0,
            cbar_kws={'label': 'Sharpe Ratio'}
        )

        # Invert the y-axis so smaller long_SMA values are at the top
        ax.invert_yaxis()

        # Find the position of the best Sharpe ratio in the heatmap
        best_y = heatmap_data.index.get_loc(best_long_sma)
        best_x = heatmap_data.columns.get_loc(best_short_sma)

        # Add a star to mark the best Sharpe ratio
        # We need to add 0.5 to center the marker in the cell
        ax.add_patch(Circle((best_x + 0.5, best_y + 0.5), 0.4, facecolor='none',
                            edgecolor='white', lw=2))
        plt.plot(best_x + 0.5, best_y + 0.5, 'w*', markersize=10)

        # Set labels and title
        plt.title(f'SMA Optimization Heatmap (Best Sharpe: {best_sharpe:.4f} at {best_short_sma}/{best_long_sma})',
                fontsize=14)
        plt.xlabel('Short SMA (days)', fontsize=12)
        plt.ylabel('Long SMA (days)', fontsize=12)

        # Rotate tick labels for better readability
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)

        # Add a text annotation for the best parameters
        plt.annotate(
            f'Best: Short={best_short_sma}, Long={best_long_sma}\nSharpe={best_sharpe:.4f}, Trades={best_trades}',
            xy=(best_x + 0.5, best_y + 0.5),
            xytext=(best_x + 5, best_y + 5),
            arrowprops=dict(arrowstyle="->", color='white'),
            color='white',
            backgroundcolor='black',
            bbox=dict(boxstyle="round,pad=0.3", fc="black", alpha=0.7)
        )

        # Display and save the plot
        plt.tight_layout()

        # Return the data and best parameters
        return data, best_short_sma, best_long_sma, best_sharpe, best_trades

    def compute_hierarchical_medoids(X, labels, valid_clusters):
        """
        Compute medoids for each valid cluster from hierarchical clustering
        A medoid is the data point in a cluster that has the minimum average distance to all other points in the cluster

        Parameters:
        X: numpy array of shape (n_samples, n_features) - Original data points (not scaled)
        labels: numpy array of shape (n_samples,) - Cluster labels for each data point
        valid_clusters: set - Set of valid cluster IDs

        Returns:
        list of tuples - Each tuple contains (short_SMA, long_SMA, sharpe_ratio, trades) for each medoid
        """
        # Initialize list to store medoids for each cluster
        medoids = []

        # Process each valid cluster
        for cluster_id in valid_clusters:
            # Extract points belonging to this cluster
            cluster_indices = np.where(labels == cluster_id)[0]
            cluster_points = X[cluster_indices]

            if len(cluster_points) == 0:
                continue

            if len(cluster_points) == 1:
                # If only one point in cluster, it's the medoid
                medoids.append(tuple(cluster_points[0]))
                continue

            # Calculate pairwise distances within cluster
            min_total_distance = float('inf')
            medoid = None

            for i, point1 in enumerate(cluster_points):
                total_distance = 0
                for j, point2 in enumerate(cluster_points):
                    # Calculate Euclidean distance between points
                    distance = np.sqrt(np.sum((point1 - point2) ** 2))
                    total_distance += distance

                if total_distance < min_total_distance:
                    min_total_distance = total_distance
                    medoid = point1

            if medoid is not None:
                medoids.append(tuple(medoid))

            # Print medoid info
            print(f"Cluster {cluster_id} medoid: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                f"Sharpe={medoid[2]:.4f}, Trades={int(medoid[3])}")

        return medoids

    def create_dendrogram(X_scaled, method='ward', figsize=(12, 8), color_threshold=None, truncate_mode=None,
                        p=10, save_path=None):
        """
        Create and display a dendrogram for hierarchical clustering

        Parameters:
        X_scaled: numpy array - Scaled data points
        method: str - Linkage method ('ward', 'complete', 'average', 'single')
        figsize: tuple - Figure size
        color_threshold: float - The threshold to apply for coloring the dendrogram
        truncate_mode: str - 'lastp' for last p branches, 'level' for no more than p levels
        p: int - Used with truncate_mode
        save_path: str - Path to save the dendrogram image

        Returns:
        Z: numpy array - The hierarchical clustering linkage matrix
        """
        # Use a sample if there are too many points
        if len(X_scaled) > 1000:
            np.random.seed(42)  # For reproducibility
            sample_indices = np.random.choice(len(X_scaled), size=1000, replace=False)
            X_sample = X_scaled[sample_indices]
            print(f"Using a random sample of 1000 points for dendrogram creation (out of {len(X_scaled)} total points)")
        else:
            X_sample = X_scaled

        # Compute the distance matrix
        dist_matrix = pdist(X_sample, metric='euclidean')

        # Compute the linkage matrix
        Z = shc.linkage(dist_matrix, method=method)

        # Print some statistics about the linkage
        print(f"\nDendrogram using {method} linkage method:")
        print(f"Number of data points: {len(X_sample)}")
        print(f"Cophenetic correlation: {shc.cophenet(Z, dist_matrix)[0]:.4f}")

        # Create a new figure
        plt.figure(figsize=figsize)

        # Set the background color
        plt.gca().set_facecolor('white')

        # Set a title
        plt.title(f'Hierarchical Clustering Dendrogram ({method} linkage)', fontsize=14)

        # Generate the dendrogram
        dendrogram = shc.dendrogram(
            Z,
            truncate_mode='lastp',  # Only show the last p merges
            p=30,  # Show only 30 merges
            leaf_rotation=90.,
            leaf_font_size=10.,
            show_contracted=True,
            color_threshold=color_threshold
        )

        # Add labels and axes
        plt.xlabel('Sample Index or Cluster Size', fontsize=12)
        plt.ylabel('Distance', fontsize=12)

        # Add a horizontal line to indicate a distance threshold if specified
        if color_threshold is not None:
            plt.axhline(y=color_threshold, color='crimson', linestyle='--',
                        label=f'Threshold: {color_threshold:.2f}')
            plt.legend()

        # Adjust layout
        plt.tight_layout()

        # Save the plot
        save_plot('Hierarchical_Dendrogram.png')

        return Z

    def hierarchical_cluster_analysis(file_path='sma_all_results.txt', min_trades=MIN_TRADES, max_trades=MAX_TRADES,
                        min_elements_per_cluster=MIN_ELEMENTS_PER_CLUSTER):
        """
        Perform hierarchical clustering analysis on SMA optimization results to find robust parameter regions
        
        Parameters:
        file_path: str - Path to the file with SMA results
        min_trades: int - Minimum number of trades to consider
        max_trades: int - Maximum number of trades to consider
        min_elements_per_cluster: int - Minimum number of elements per cluster
        
        Returns:
        tuple - (X_filtered_full, medoids, top_medoids, max_sharpe_point, labels)
        """
        print(f"\n----- HIERARCHICAL CLUSTER ANALYSIS -----")
        print(f"Loading data from {file_path}...")

        # Load the data
        df = pd.read_csv(file_path)

        # Convert data to numpy array for easier processing
        X_full = df[['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

        # STEP 1: First filter by trades and valid SMA combinations
        X_filtered_full = X_full[(X_full[:, 0] < X_full[:, 1]) &  # short_SMA < long_SMA
                    (X_full[:, 3] >= min_trades) &  # trades >= min_trades
                    (X_full[:, 3] <= max_trades)]  # trades <= max_trades

        if len(X_filtered_full) == 0:
            print(f"No data points meet the criteria after filtering! Adjust min_trades ({min_trades}) and max_trades ({max_trades}).")
            return None, None, None, None, None

        # STEP 2: Use exactly 3 dimensions for clustering - same as K-means
        X_filtered = X_filtered_full[:, 0:3]  # Only short_SMA, long_SMA, and sharpe_ratio
        
        print(f"Filtered data to {len(X_filtered)} points with {min_trades}-{max_trades} trades")

        # Extract the fields for better scaling visibility
        short_sma_values = X_filtered[:, 0]
        long_sma_values = X_filtered[:, 1]
        sharpe_values = X_filtered[:, 2]
        trades_values = X_filtered_full[:, 3]

        print(f"Short SMA range: {short_sma_values.min()} to {short_sma_values.max()}")
        print(f"Long SMA range: {long_sma_values.min()} to {long_sma_values.max()}")
        print(f"Sharpe ratio range: {sharpe_values.min():.4f} to {sharpe_values.max():.4f}")
        print(f"Trades range: {trades_values.min()} to {trades_values.max()}")

        # Scale the data for clustering - using StandardScaler for each dimension
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_filtered)  # Only scale the 3 dimensions we use for clustering

        # Print scaling info for verification
        print("\nScaled data information:")
        scaled_short = X_scaled[:, 0]
        scaled_long = X_scaled[:, 1]
        scaled_sharpe = X_scaled[:, 2]

        print(f"Scaled Short SMA range: {scaled_short.min():.4f} to {scaled_short.max():.4f}")
        print(f"Scaled Long SMA range: {scaled_long.min():.4f} to {scaled_long.max():.4f}")
        print(f"Scaled Sharpe ratio range: {scaled_sharpe.min():.4f} to {scaled_sharpe.max():.4f}")

        # Create a dendrogram to help visualize (optional, can be commented out)
        print("\nCreating dendrogram to visualize hierarchical structure...")
        linkage_method = 'ward'  # Ward minimizes the variance within clusters
        Z = create_dendrogram(X_scaled, method=linkage_method, figsize=(12, 8))

        # STEP 3: Determine number of clusters - use the same DEFAULT_NUM_CLUSTERS as K-means
        print(f"Using default number of clusters: {DEFAULT_NUM_CLUSTERS}")
        k = DEFAULT_NUM_CLUSTERS

        # STEP 4: Apply hierarchical clustering
        print(f"Performing hierarchical clustering with {k} clusters using {linkage_method} linkage...")
        hierarchical = AgglomerativeClustering(
            n_clusters=k,
            linkage=linkage_method
        )

        # Fit the model and get cluster labels
        labels = hierarchical.fit_predict(X_scaled)

        # STEP 5: Count elements per cluster
        unique_labels, counts = np.unique(labels, return_counts=True)
        cluster_sizes = dict(zip(unique_labels, counts))

        print("\nCluster sizes:")
        for cluster_id, size in cluster_sizes.items():
            print(f"Cluster {cluster_id}: {size} elements")

        # STEP 6: Filter clusters with enough elements - SAME as K-means approach
        valid_clusters = {i for i, count in cluster_sizes.items() if count >= min_elements_per_cluster}
        
        if not valid_clusters:
            print(f"No clusters have at least {min_elements_per_cluster} elements! Using all clusters.")
            valid_clusters = set(unique_labels)
        
        # Don't filter the data points here yet - just keep track of which clusters are valid

        # STEP 7: Compute medoids using the original data (not scaled)
        print("Computing medoids for each cluster...")
        medoids = compute_hierarchical_medoids(X_filtered_full, labels, valid_clusters)

        # Find max Sharpe ratio point overall
        max_sharpe_idx = np.argmax(df['sharpe_ratio'].values)
        max_sharpe_point = df.iloc[max_sharpe_idx][['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades']].values

        # STEP 8: Sort medoids by Sharpe ratio
        medoids_sorted = sorted(medoids, key=lambda x: float(x[2]), reverse=True)
        top_medoids = medoids_sorted[:5]  # Get top 5 medoids by Sharpe ratio

        # Print results
        print("\n----- HIERARCHICAL CLUSTERING RESULTS -----")
        print(f"Max Sharpe point: Short SMA={int(max_sharpe_point[0])}, Long SMA={int(max_sharpe_point[1])}, "
            f"Sharpe={max_sharpe_point[2]:.4f}, Trades={int(max_sharpe_point[3])}")

        print("\nTop 5 Medoids (by Sharpe ratio):")
        for idx, medoid in enumerate(top_medoids, 1):
            print(f"Top {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

        # Create visualization - pass all the original filtered data and valid_clusters
        create_hierarchical_cluster_visualization(X_filtered_full, medoids, top_medoids, max_sharpe_point, labels, valid_clusters)
        
        return X_filtered_full, medoids, top_medoids, max_sharpe_point, labels

    def plot_strategy_performance(short_sma, long_sma, top_medoids=None, big_point_value=1, slippage=0, capital=1000000, atr_period=14):
        print(f"\n----- PLOTTING STRATEGY PERFORMANCE -----")
        print(f"Using Short SMA: {short_sma}, Long SMA: {long_sma}")
        print(f"Trading with ATR-based position sizing (Capital: ${capital:,}, ATR Period: {atr_period})")
        if top_medoids:
            print(f"Including top {len(top_medoids)} medoids")

        # Load data from local file
        print(f"Loading {TICKER} data from local files...")
        data_file = find_futures_file(SYMBOL, DATA_DIR)
        if not data_file:
            print(f"Error: No data file found for {TICKER} in {DATA_DIR}")
            exit(1)

        print(f"Found data file: {os.path.basename(data_file)}")
        print(f"File size: {os.path.getsize(data_file)} bytes")

        # Load the data from local file
        all_data = read_ts.read_ts_ohlcv_dat(data_file)
        data_obj = all_data[0]
        ohlc_data = data_obj.data.copy()

        # Convert the OHLCV data to the format expected by the strategy
        data = ohlc_data.rename(columns={
            'datetime': 'Date',
            'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close': 'Close',
            'volume': 'Volume'
        })
        data.set_index('Date', inplace=True)

        # Add a warm-up period before the start date
        original_start_idx = None
        if START_DATE and END_DATE:
            # Calculate warm-up period for SMA calculation (longest SMA + buffer)
            warm_up_days = max(short_sma, long_sma) * 3  # Use 3x the longest SMA as warm-up
            
            # Convert dates to datetime
            start_date = pd.to_datetime(START_DATE)
            end_date = pd.to_datetime(END_DATE)
            
            # Adjust start date for warm-up
            adjusted_start = start_date - pd.Timedelta(days=warm_up_days)
            
            # Filter with the extended date range
            extended_data = data[(data.index >= adjusted_start) & (data.index <= end_date)]
            
            # Store the index where the actual analysis should start
            if not extended_data.empty:
                # Find the closest index to our original start date
                original_start_idx = extended_data.index.get_indexer([start_date], method='nearest')[0]
            
            print(f"Added {warm_up_days} days warm-up period before {START_DATE}")
            data = extended_data
        
        # Create a dictionary to store results for each strategy
        strategies = {
            'Best': {'short_sma': short_sma, 'long_sma': long_sma}
        }

        # Add medoids in their original order, including original Sharpe and Trades
        if top_medoids:
            print("\nUSING THESE MEDOIDS (IN ORIGINAL ORDER):")
            for i, medoid in enumerate(top_medoids, 1):
                strategies[f'Medoid {i}'] = {
                    'short_sma': int(medoid[0]),
                    'long_sma': int(medoid[1]),
                    'original_sharpe': float(medoid[2]),  # Store the original Sharpe ratio
                    'original_trades': int(medoid[3])     # Store the original number of trades
                }
                print(f"Medoid {i}: SMA({int(medoid[0])}/{int(medoid[1])}) - Original Sharpe: {float(medoid[2]):.4f}, Trades: {int(medoid[3])}")

        # Apply the proper strategy for each parameter set
        for name, params in strategies.items():
            # Calculate centered SMAs directly
            data[f'SMA_Short_{name}'] = data['Close'].rolling(window=params['short_sma'], center=True).mean()
            data[f'SMA_Long_{name}'] = data['Close'].rolling(window=params['long_sma'], center=True).mean()
            
            # Create a strategy instance for each parameter set
            sma_strategy = SMAStrategy(
                short_sma=params['short_sma'],
                long_sma=params['long_sma'],
                big_point_value=big_point_value,
                slippage=slippage,
                capital=capital,
                atr_period=atr_period
            )

            # Apply the strategy
            data = sma_strategy.apply_strategy(
                data.copy(),
                strategy_name=name
            )

        # Trim data to the original date range if we added warm-up period
        if original_start_idx is not None:
            data_for_evaluation = data.iloc[original_start_idx:]
            print(f"Trimmed warm-up period. Original data length: {len(data)}, Evaluation data length: {len(data_for_evaluation)}")
        else:
            data_for_evaluation = data
        
        # Calculate split index for in-sample/out-of-sample using the trimmed data
        split_index = int(len(data_for_evaluation) * TRAIN_TEST_SPLIT)
        split_date = data_for_evaluation.index[split_index]

        # Create color palette for strategies
        colors = ['blue', 'green', 'purple', 'orange', 'brown', 'pink']

        # Create the performance visualization with just two panels
        plt.figure(figsize=(14, 12))

        # Plot price and SMA (first subplot)
        plt.subplot(2, 1, 1)
        plt.plot(data_for_evaluation.index, data_for_evaluation['Close'], label=f'{SYMBOL} Price', color='black', alpha=0.5)
        
        # Use the centered SMAs for plotting
        for name, params in strategies.items():
            if name == 'Best':
                plt.plot(data_for_evaluation.index, data_for_evaluation[f'SMA_Short_{name}'], 
                        label=f'Short SMA ({params["short_sma"]})', color='orange')
                plt.plot(data_for_evaluation.index, data_for_evaluation[f'SMA_Long_{name}'], 
                        label=f'Long SMA ({params["long_sma"]})', color='blue')
        
        # Mark the train/test split
        plt.axvline(x=split_date, color='black', linestyle='--', alpha=0.7)
        
        plt.legend(loc='upper left')
        plt.title(f'{SYMBOL} Price and SMA Indicators')
        plt.ylabel('Price')
        plt.grid(True, alpha=0.3)

        # Plot cumulative P&L (second subplot)
        plt.subplot(2, 1, 2)

        for i, (name, params) in enumerate(strategies.items()):
            color = colors[i % len(colors)]

            # Plot full period P&L
            plt.plot(data_for_evaluation.index, data_for_evaluation[f'Cumulative_PnL_{name}'],
                    label=f'{name} ({params["short_sma"]}/{params["long_sma"]})', color=color)

            # Plot out-of-sample portion with thicker line
            plt.plot(data_for_evaluation.index[split_index:], data_for_evaluation[f'Cumulative_PnL_{name}'].iloc[split_index:],
                    color=color, linewidth=2.5, alpha=0.7)

        plt.axvline(x=split_date, color='black', linestyle='--',
                    label=f'Train/Test Split ({int(TRAIN_TEST_SPLIT * 100)}%/{int((1 - TRAIN_TEST_SPLIT) * 100)}%)')
        plt.axhline(y=0.0, color='gray', linestyle='-', alpha=0.5, label='Break-even')
        plt.legend(loc='upper left')
        plt.title('Strategy Cumulative P&L')
        plt.ylabel('P&L ($)')
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        save_plot('Hierarchical_Strategy_Performance.png')

        # Create a list to store performance data for saving to file
        performance_data = []

        # Print detailed performance metrics for all strategies with in-sample and out-of-sample breakdown
        print("\n----- PERFORMANCE SUMMARY -----")

        # IN-SAMPLE PERFORMANCE
        print("\nIN-SAMPLE PERFORMANCE:")
        header = f"{'Strategy':<10} | {'Short/Long':<10} | {'P&L':>12} | {'Sharpe':>7} | {'Trades':>6}"
        separator = "-" * len(header)
        print(separator)
        print(header)
        print(separator)

        # Calculate in-sample metrics using trimmed data
        for name, params in strategies.items():
            short = params['short_sma']
            long = params['long_sma']

            # Get in-sample data from trimmed dataset
            in_sample = data_for_evaluation.iloc[:split_index]

            # Calculate in-sample metrics
            in_sample_daily_pnl = in_sample[f'Daily_PnL_{name}']
            in_sample_cumulative_pnl = in_sample[f'Daily_PnL_{name}'].sum()
            
            # Calculate average position size for in-sample
            avg_pos_size = in_sample[f'Position_Size_{name}'].mean()

            # Use original Sharpe and Trades for medoids, calculate for Best
            if name.startswith('Medoid'):
                in_sample_sharpe = params['original_sharpe']
                in_sample_trades = params['original_trades']
            else:
                # Calculate Sharpe ratio (annualized) for Best
                if in_sample_daily_pnl.std() > 0:
                    in_sample_sharpe = in_sample_daily_pnl.mean() / in_sample_daily_pnl.std() * np.sqrt(252)
                else:
                    in_sample_sharpe = 0
                # Count in-sample trades for Best
                in_sample_trades = in_sample[f'Position_Change_{name}'].sum()
            
            # Print in the original order
            row = f"{name:<10} | {short:>4}/{long:<5} | ${in_sample_cumulative_pnl:>10,.2f} | {in_sample_sharpe:>6.3f} | {in_sample_trades:>6}"
            print(row)
            
            # Store data for later use
            performance_data.append({
                'Period': 'In-Sample',
                'Strategy': name,
                'Short_SMA': short,
                'Long_SMA': long,
                'PnL': in_sample_cumulative_pnl,
                'Avg_Position': avg_pos_size,
                'Sharpe': in_sample_sharpe,
                'Trades': in_sample_trades
            })

        print(separator)

        # OUT-OF-SAMPLE PERFORMANCE
        print("\nOUT-OF-SAMPLE PERFORMANCE:")
        print(separator)
        print(header)
        print(separator)

        # Calculate out-of-sample metrics using trimmed data
        for name, params in strategies.items():
            short = params['short_sma']
            long = params['long_sma']

            # Get out-of-sample data from trimmed dataset
            out_sample = data_for_evaluation.iloc[split_index:]

            # Calculate out-of-sample metrics
            out_sample_daily_pnl = out_sample[f'Daily_PnL_{name}']
            out_sample_cumulative_pnl = out_sample[f'Daily_PnL_{name}'].sum()
            
            # Calculate average position size for out-of-sample
            avg_pos_size = out_sample[f'Position_Size_{name}'].mean()

            # Calculate Sharpe ratio (annualized)
            if out_sample_daily_pnl.std() > 0:
                out_sample_sharpe = out_sample_daily_pnl.mean() / out_sample_daily_pnl.std() * np.sqrt(252)
            else:
                out_sample_sharpe = 0

            # Count out-of-sample trades
            out_sample_trades = out_sample[f'Position_Change_{name}'].sum()
            
            # Print in the original order
            row = f"{name:<10} | {short:>4}/{long:<5} | ${out_sample_cumulative_pnl:>10,.2f} | {out_sample_sharpe:>6.3f} | {out_sample_trades:>6}"
            print(row)
            
            # Store data for later use
            performance_data.append({
                'Period': 'Out-of-Sample',
                'Strategy': name,
                'Short_SMA': short,
                'Long_SMA': long,
                'PnL': out_sample_cumulative_pnl,
                'Avg_Position': avg_pos_size,
                'Sharpe': out_sample_sharpe,
                'Trades': out_sample_trades
            })

        print(separator)

        # FULL PERIOD PERFORMANCE
        print("\nFULL PERIOD PERFORMANCE:")
        print(separator)
        print(header)
        print(separator)

        # Calculate full period metrics using trimmed data
        for name, params in strategies.items():
            short = params['short_sma']
            long = params['long_sma']

            # Calculate full period metrics using trimmed data
            full_daily_pnl = data_for_evaluation[f'Daily_PnL_{name}']
            full_cumulative_pnl = full_daily_pnl.sum()
            
            # Calculate average position size for full period
            avg_pos_size = data_for_evaluation[f'Position_Size_{name}'].mean()
            max_pos_size = data_for_evaluation[f'Position_Size_{name}'].max()

            # Calculate Sharpe ratio (annualized)
            if full_daily_pnl.std() > 0:
                full_sharpe = full_daily_pnl.mean() / full_daily_pnl.std() * np.sqrt(252)
            else:
                full_sharpe = 0

            # Count full period trades
            full_trades = data_for_evaluation[f'Position_Change_{name}'].sum()
            
            # Print in the original order
            row = f"{name:<10} | {short:>4}/{long:<5} | ${full_cumulative_pnl:>10,.2f} | {full_sharpe:>6.3f} | {full_trades:>6}"
            print(row)
            
            # Store data for later use
            performance_data.append({
                'Period': 'Full',
                'Strategy': name,
                'Short_SMA': short,
                'Long_SMA': long,
                'PnL': full_cumulative_pnl,
                'Avg_Position': avg_pos_size,
                'Max_Position': max_pos_size,
                'Sharpe': full_sharpe,
                'Trades': full_trades
            })
            
            # Additional metrics for the best strategy
            if name == 'Best':
                print(f"\nAdditional metrics for Best strategy:")
                print(f"Maximum position size: {max_pos_size:.2f} contracts")
                print(f"Average ATR value: {data_for_evaluation['ATR_Best'].mean():.4f}")
                
                # Calculate drawdown using trimmed data
                peak = data_for_evaluation[f'Cumulative_PnL_{name}'].cummax()
                drawdown = data_for_evaluation[f'Cumulative_PnL_{name}'] - peak
                max_drawdown = drawdown.min()
                
                print(f"Maximum drawdown: ${max_drawdown:.2f}")
                
                # Calculate win rate using trimmed data
                daily_win_rate = (data_for_evaluation[f'Daily_PnL_{name}'] > 0).mean() * 100
                print(f"Daily win rate: {daily_win_rate:.2f}%")

        print(separator)

        # Save performance data to CSV in the ticker output directory
        performance_df = pd.DataFrame(performance_data)
        save_results(performance_df, f"{SYMBOL}_hierarchical_performance_summary.csv")

        return data_for_evaluation  # Return the trimmed data instead of full data

    def bimonthly_out_of_sample_comparison(data, best_short_sma, best_long_sma, top_medoids, min_sharpe=0.2, 
                                big_point_value=big_point_value, slippage=slippage,
                                capital=capital, atr_period=atr_period):
        """
        Compare bimonthly (2-month) performance between the best Sharpe strategy and a portfolio of top medoids
        using ATR-based position sizing.
        
        Parameters:
        data: DataFrame with market data (should already be trimmed to exclude warmup period)
        best_short_sma: int - The short SMA period for the best Sharpe strategy
        best_long_sma: int - The long SMA period for the best Sharpe strategy
        top_medoids: list - List of top medoids, each as (short_sma, long_sma, sharpe, trades)
        min_sharpe: float - Minimum Sharpe ratio threshold for medoids to be included
        big_point_value: float - Big point value for the futures contract
        slippage: float - Slippage value in price units
        capital: float - Capital allocation for position sizing
        atr_period: int - Period for ATR calculation
        """
        print(f"\n----- BIMONTHLY OUT-OF-SAMPLE COMPARISON -----")
        print(f"Best Sharpe: ({best_short_sma}/{best_long_sma})")
        print(f"Using ATR-based position sizing (Capital: ${capital:,}, ATR Period: {atr_period})")
        
        # Handle the case where top_medoids is None
        if top_medoids is None:
            print("No medoids provided. Comparison cannot be performed.")
            return None
        
        # ADDED: Ensure we have trimmed data
        if data is None:
            # Load data from local file
            print(f"Loading {TICKER} data from local files...")
            data_file = find_futures_file(SYMBOL, DATA_DIR)
            if not data_file:
                print(f"Error: No data file found for {TICKER} in {DATA_DIR}")
                exit(1)
            
            print(f"Found data file: {os.path.basename(data_file)}")
            print(f"File size: {os.path.getsize(data_file)} bytes")

            # Load the data from local file
            all_data = read_ts.read_ts_ohlcv_dat(data_file)
            data_obj = all_data[0]
            ohlc_data = data_obj.data.copy()

            # Convert the OHLCV data to the format expected by the strategy
            data = ohlc_data.rename(columns={
                'datetime': 'Date',
                'open': 'Open',
                'high': 'High',
                'low': 'Low',
                'close': 'Close',
                'volume': 'Volume'
            })
            data.set_index('Date', inplace=True)
            
            # Add a warm-up period before the start date
            original_start_idx = None
            if START_DATE and END_DATE:
                # Calculate warm-up period for SMA calculation (longest SMA + buffer)
                warm_up_days = max(best_short_sma, best_long_sma) * 3  # Use 3x the longest SMA as warm-up
                
                # Convert dates to datetime
                start_date = pd.to_datetime(START_DATE)
                end_date = pd.to_datetime(END_DATE)
                
                # Adjust start date for warm-up
                adjusted_start = start_date - pd.Timedelta(days=warm_up_days)
                
                # Filter with the extended date range
                extended_data = data[(data.index >= adjusted_start) & (data.index <= end_date)]
                
                # Store the index where the actual analysis should start
                if not extended_data.empty:
                    # Find the closest index to our original start date
                    original_start_idx = extended_data.index.get_indexer([start_date], method='nearest')[0]
                
                print(f"Added {warm_up_days} days warm-up period before {START_DATE}")
                data = extended_data
                
                # Trim data to the original date range
                if original_start_idx is not None:
                    data = data.iloc[original_start_idx:]
                    print(f"Trimmed warm-up period. Data length: {len(data)}")
        
        # The top_medoids are already sorted by Sharpe ratio in cluster_analysis
        # Just print them for verification
        print("\nUSING TOP MEDOIDS (BY SHARPE RATIO):")
        for i, medoid in enumerate(top_medoids, 1):
            print(f"Medoid {i}: ({int(medoid[0])}/{int(medoid[1])}) - Sharpe: {float(medoid[2]):.4f}, Trades: {int(medoid[3])}")
        
        # Take at most 3 medoids and filter by minimum Sharpe
        filtered_medoids = []
        for i, m in enumerate(top_medoids[:3]):
            # Check if we can access the required elements
            try:
                # Extract Sharpe ratio and check if it meets the threshold
                short_sma = m[0]
                long_sma = m[1]
                sharpe = float(m[2])  # Convert to float to handle numpy types
                trades = m[3]
                
                if sharpe >= min_sharpe:
                    filtered_medoids.append(m)
                    print(f"Selected medoid {i+1} with Sharpe {sharpe:.4f}")
            except (IndexError, TypeError) as e:
                print(f"Error processing medoid: {e}")
        
        if not filtered_medoids:
            print(f"No medoids have a Sharpe ratio >= {min_sharpe}. Comparison cannot be performed.")
            return None
        
        print(f"Creating portfolio of {len(filtered_medoids)} medoids with Sharpe ratio >= {min_sharpe}:")
        for i, medoid in enumerate(filtered_medoids, 1):
            print(f"Final Medoid {i}: ({int(medoid[0])}/{int(medoid[1])}) - Sharpe: {float(medoid[2]):.4f}")
        
        # Create strategies
        strategies = {
            'Best': {'short_sma': best_short_sma, 'long_sma': best_long_sma}
        }
        
        # Add filtered medoids
        for i, medoid in enumerate(filtered_medoids, 1):
            strategies[f'Medoid_{i}'] = {'short_sma': int(medoid[0]), 'long_sma': int(medoid[1])}
        
        # Apply each strategy to the data
        for name, params in strategies.items():
            strategy = SMAStrategy(
                short_sma=params['short_sma'],
                long_sma=params['long_sma'],
                big_point_value=big_point_value,
                slippage=slippage,
                capital=capital,
                atr_period=atr_period
            )
            
            # Apply the strategy
            data = strategy.apply_strategy(
                data.copy(),
                strategy_name=name
            )
        
        # Get the out-of-sample split date
        split_index = int(len(data) * TRAIN_TEST_SPLIT)
        split_date = data.index[split_index]
        print(f"Out-of-sample period starts on: {split_date.strftime('%Y-%m-%d')}")
        
        # Get out-of-sample data
        oos_data = data.iloc[split_index:].copy()
        
        # Add a year and bimonthly period columns for grouping (each year has 6 bimonthly periods)
        oos_data['year'] = oos_data.index.year.astype(int)
        oos_data['bimonthly'] = ((oos_data.index.month - 1) // 2 + 1).astype(int)
        
        # Create simplified period labels with just the start month (YYYY-MM)
        oos_data['period_label'] = oos_data.apply(
            lambda row: f"{int(row['year'])}-{int((row['bimonthly'] - 1) * 2 + 1):02d}",
            axis=1
        )
        
        # Create a DataFrame to store bimonthly Sharpe ratios
        bimonthly_sharpe = []
        
        # Group by year and bimonthly period, calculate Sharpe ratio for each period
        for period_label, group in oos_data.groupby('period_label'):
            # Skip periods with too few trading days
            if len(group) < 10:
                continue
                
            # Create a bimonthly result entry
            year, start_month = period_label.split('-')
            year = int(year)
            start_month = int(start_month)
            
            bimonthly_result = {
                'period_label': period_label,
                'date': pd.Timestamp(year=year, month=start_month, day=15),  # Middle of first month in period
                'trading_days': len(group),
            }
            
            # Calculate Sharpe ratio for each strategy in this period
            for name in strategies.keys():
                # Get returns for this strategy in this period
                returns = group[f'Daily_PnL_{name}']
                
                # Calculate Sharpe ratio (annualized)
                if len(returns) > 1 and returns.std() > 0:
                    sharpe = returns.mean() / returns.std() * np.sqrt(252)
                else:
                    sharpe = 0
                    
                bimonthly_result[f'{name}_sharpe'] = sharpe
                bimonthly_result[f'{name}_return'] = returns.sum()  # Total P&L for the period
            
            # Calculate the normalized average of medoid Sharpe ratios
            medoid_sharpes = [bimonthly_result[f'Medoid_{i}_sharpe'] for i in range(1, len(filtered_medoids) + 1)]
            bimonthly_result['Avg_Medoid_sharpe'] = sum(medoid_sharpes) / len(filtered_medoids)
            
            # Calculate the normalized average of medoid returns
            medoid_returns = [bimonthly_result[f'Medoid_{i}_return'] for i in range(1, len(filtered_medoids) + 1)]
            bimonthly_result['Avg_Medoid_return'] = sum(medoid_returns) / len(filtered_medoids)
            
            bimonthly_sharpe.append(bimonthly_result)
        
        # Convert to DataFrame
        bimonthly_sharpe_df = pd.DataFrame(bimonthly_sharpe)
        
        # Sort the DataFrame by date for proper chronological display
        if not bimonthly_sharpe_df.empty:
            bimonthly_sharpe_df = bimonthly_sharpe_df.sort_values('date')
        else:
            print(f"WARNING: No bimonthly periods found for {SYMBOL}. Cannot create chart.")
            return None
        
        # Add rounded values to dataframe for calculations
        bimonthly_sharpe_df['Best_sharpe_rounded'] = np.round(bimonthly_sharpe_df['Best_sharpe'], 2)
        bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] = np.round(bimonthly_sharpe_df['Avg_Medoid_sharpe'], 2)
        
        # Print detailed comparison of Sharpe ratios
        print("\nDetailed Sharpe ratio comparison by period:")
        print(f"{'Period':<12} | {'Best Sharpe':>12} | {'Medoid Portfolio':>16} | {'Difference':>12} | {'Portfolio Wins':<14}")
        print("-" * 80)
        
        for idx, row in bimonthly_sharpe_df.iterrows():
            period = row['period_label']
            best_sharpe = row['Best_sharpe']
            avg_medoid_sharpe = row['Avg_Medoid_sharpe']
            best_rounded = row['Best_sharpe_rounded']
            avg_medoid_rounded = row['Avg_Medoid_sharpe_rounded']
            
            diff = avg_medoid_sharpe - best_sharpe
            portfolio_wins = avg_medoid_sharpe > best_sharpe
            
            print(f"{period:<12} | {best_sharpe:12.6f} | {avg_medoid_sharpe:16.6f} | {diff:12.6f} | {portfolio_wins!s:<14}")
        
        # Calculate win rate using raw values
        portfolio_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe'] > bimonthly_sharpe_df['Best_sharpe'])
        total_periods = len(bimonthly_sharpe_df)
        win_percentage = (portfolio_wins / total_periods) * 100 if total_periods > 0 else 0
        
        # Calculate win rate using rounded values (for alternative comparison)
        rounded_wins = sum(bimonthly_sharpe_df['Avg_Medoid_sharpe_rounded'] > bimonthly_sharpe_df['Best_sharpe_rounded'])
        rounded_win_percentage = (rounded_wins / total_periods) * 100 if total_periods > 0 else 0
        
        print(f"\nBimonthly periods analyzed: {total_periods}")
        print(f"Medoid Portfolio Wins: {portfolio_wins} of {total_periods} periods ({win_percentage:.2f}%)")
        print(f"Using rounded values (2 decimal places): {rounded_wins} of {total_periods} periods ({rounded_win_percentage:.2f}%)")
        
        # Create a bar plot to compare bimonthly Sharpe ratios
        plt.figure(figsize=(14, 8))
        
        # Set up x-axis dates
        x = np.arange(len(bimonthly_sharpe_df))
        width = 0.35  # Width of the bars
        
        # Create bars
        plt.bar(x - width/2, bimonthly_sharpe_df['Best_sharpe'], width, 
            label=f'Best Sharpe ({best_short_sma}/{best_long_sma})', color='blue')
        plt.bar(x + width/2, bimonthly_sharpe_df['Avg_Medoid_sharpe'], width, 
            label=f'Medoid Portfolio ({len(filtered_medoids)} strategies)', color='green')
        
        # Add a horizontal line at Sharpe = 0
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        
        # Create medoid description for the title
        medoid_desc = ", ".join([f"({int(m[0])}/{int(m[1])})" for m in filtered_medoids])
        
        # Customize the plot - using rounded win percentage instead of raw
        plt.title(f'Bimonthly Sharpe Ratio Comparison (Out-of-Sample Period)\n' + 
                f'Medoid Portfolio [{medoid_desc}] outperformed {rounded_win_percentage:.2f}% of the time', 
                fontsize=14)
        plt.xlabel('Bimonthly Period (Start Month)', fontsize=12)
        plt.ylabel('Sharpe Ratio (Annualized)', fontsize=12)
        
        # Simplified x-tick labels with just the period start month
        plt.xticks(x, bimonthly_sharpe_df['period_label'], rotation=45)
        
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Create legend with both strategies - moved to bottom to avoid overlap with title
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2,
                frameon=True, fancybox=True, framealpha=0.9, fontsize=10)
        
        # Add a text box with rounded win percentage instead of raw - moved to right side
        plt.annotate(f'Medoid Portfolio Win Rate: {rounded_win_percentage:.2f}%\n'
                    f'({rounded_wins} out of {total_periods} periods)\n'
                    f'Portfolio: {len(filtered_medoids)} medoids with Sharpe ≥ {min_sharpe}\n'
                    f'ATR-Based Position Sizing (${capital:,}, {atr_period} days)',
                    xy=(0.7, 0.95), xycoords='axes fraction',
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
                    fontsize=12)
        
        plt.tight_layout(rect=[0, 0, 1, 0.95])  # Add extra space at the bottom for the legend
        save_plot('Hierarchical_Bimonthly_Comparison.png')
        
        # Save bimonthly comparison to CSV
        save_results(bimonthly_sharpe_df, f"{SYMBOL}_hierarchical_bimonthly_comparison.csv")
        
        # Save win percentage and medoid parameters to Excel file
        try:
            import openpyxl
            
            # Path to the Excel file
            excel_file = r"C:\Users\Admin\Documents\darbas\Results.xlsx"
            
            # Check if file exists
            if not os.path.exists(excel_file):
                print(f"Excel file not found at: {excel_file}")
                return bimonthly_sharpe_df
                
            print(f"Updating Excel file with Hierarchical results for {SYMBOL}...")
            
            # Load the workbook
            wb = openpyxl.load_workbook(excel_file)
            
            # Get the active sheet
            sheet = wb.active
            
            # Find the row with the ticker symbol or the first empty row
            row = 3  # Start from row 3 (assuming rows 1-2 have headers)
            ticker_row = None
            
            while True:
                cell_value = sheet.cell(row=row, column=1).value
                if cell_value == SYMBOL:
                    # Found the ticker symbol
                    ticker_row = row
                    break
                elif cell_value is None:
                    # Found an empty row
                    ticker_row = row
                    # Write the ticker symbol in column A
                    sheet.cell(row=ticker_row, column=1).value = SYMBOL
                    break
                row += 1
            
            # Round the win percentage to one decimal place
            rounded_win_percentage_1dp = round(rounded_win_percentage, 1)
            
            # Write the win percentage in column C (Hierarchy)
            sheet.cell(row=ticker_row, column=3).value = rounded_win_percentage_1dp
            
            # Write the medoid parameters in the respective cluster columns
            # Hierarchy clusters start at column I (9)
            for i, medoid in enumerate(filtered_medoids):
                if i >= 3:  # Only use up to 3 clusters
                    break
                    
                # Calculate column index: I=9, J=10, K=11 for Cluster1, Cluster2, Cluster3
                column_idx = 9 + i
                
                # Format as "short/long" (e.g., "5/20")
                param_value = f"{int(medoid[0])}/{int(medoid[1])}"
                
                # Write to Excel
                sheet.cell(row=ticker_row, column=column_idx).value = param_value
            
            # Write the best Sharpe parameters in column M (13)
            best_sharpe_params = f"{best_short_sma}/{best_long_sma}"
            sheet.cell(row=ticker_row, column=13).value = best_sharpe_params
            
            # Save the workbook
            wb.save(excel_file)
            
            print(f"Excel file updated successfully. Added {SYMBOL} with Hierarchical win rate {rounded_win_percentage_1dp}% in row {ticker_row}")
            print(f"Added best Sharpe parameters {best_sharpe_params} in column M")
            
        except Exception as e:
            print(f"Error updating Excel file: {e}")
        
        # Return the bimonthly Sharpe ratio data
        return bimonthly_sharpe_df

    def create_hierarchical_cluster_visualization(X_filtered_full, medoids, top_medoids, max_sharpe_point, labels, valid_clusters):
        """
        Create a continuous heatmap visualization with hierarchical cluster centers overlaid.
        
        Parameters:
        X_filtered_full: array - All filter-compliant data points with shape (n_samples, 4)
        medoids: list - List of medoids from valid clusters
        top_medoids: list - List of top medoids by Sharpe ratio
        max_sharpe_point: array - Point with maximum Sharpe ratio
        labels: array - Cluster labels for each point in X_filtered_full
        valid_clusters: set - Set of valid cluster IDs that meet the min_elements_per_cluster requirement
        """
        print("Creating hierarchical cluster visualization...")

        # Load the full dataset for pivot table creation
        data = pd.read_csv('sma_all_results.txt')
        
        # Create filtered dataframe from X_filtered_full but include ALL points
        # We will NOT filter by valid_clusters yet - this matches the K-means approach
        filtered_df = pd.DataFrame(X_filtered_full, columns=['short_SMA', 'long_SMA', 'sharpe_ratio', 'trades'])
        
        # Create a pivot table for the heatmap using ALL filtered data points (not just valid clusters)
        heatmap_data = filtered_df.pivot_table(
            index='long_SMA',
            columns='short_SMA',
            values='sharpe_ratio',
            fill_value=np.nan  # Use NaN for empty cells
        )

        # Create the heatmap visualization
        plt.figure(figsize=(12, 10))

        # Create mask for invalid combinations (where short_SMA >= long_SMA)
        # and also for NaN values (which represent filtered-out points)
        mask = np.zeros_like(heatmap_data, dtype=bool)
        for i, long_sma in enumerate(heatmap_data.index):
            for j, short_sma in enumerate(heatmap_data.columns):
                if short_sma >= long_sma or np.isnan(heatmap_data.iloc[i, j]):
                    mask[i, j] = True

        # Plot the base heatmap with ALL filtered data
        ax = sns.heatmap(
            heatmap_data,
            mask=mask,
            cmap='coolwarm',  # Blue to red colormap
            annot=False,      # Don't annotate each cell with its value
            fmt='.4f',
            linewidths=0,
            cbar_kws={'label': 'Sharpe Ratio'}
        )

        # Invert the y-axis so smaller long_SMA values are at the top
        ax.invert_yaxis()

        # Plot max Sharpe point (Green Star)
        try:
            best_x_pos = np.where(heatmap_data.columns == max_sharpe_point[0])[0][0] + 0.5
            best_y_pos = np.where(heatmap_data.index == max_sharpe_point[1])[0][0] + 0.5
            plt.scatter(best_x_pos, best_y_pos, marker='*', color='lime', s=200,
                        edgecolor='black', zorder=5)
        except IndexError:
            print(f"Warning: Max Sharpe point at ({max_sharpe_point[0]}, {max_sharpe_point[1]}) not found in heatmap coordinates")

        # Only plot medoids (these are already from valid clusters)
        if medoids:
            for medoid in medoids:
                try:
                    x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
                    y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
                    plt.scatter(x_pos, y_pos, marker='s', color='black', s=75, zorder=4)
                except IndexError:
                    print(f"Warning: Medoid at ({medoid[0]}, {medoid[1]}) not found in heatmap coordinates")

        # Plot top 5 medoids (Purple Diamonds)
        if top_medoids:
            for medoid in top_medoids:
                try:
                    x_pos = np.where(heatmap_data.columns == medoid[0])[0][0] + 0.5
                    y_pos = np.where(heatmap_data.index == medoid[1])[0][0] + 0.5
                    plt.scatter(x_pos, y_pos, marker='D', color='purple', s=100, zorder=5)
                except IndexError:
                    print(f"Warning: Top medoid at ({medoid[0]}, {medoid[1]}) not found in heatmap coordinates")

        # Create custom legend
        max_sharpe_handle = mlines.Line2D([], [], color='lime', marker='*', linestyle='None',
                                        markersize=15, markeredgecolor='black', label='Max Sharpe')
        medoid_handle = mlines.Line2D([], [], color='black', marker='s', linestyle='None',
                                    markersize=10, label='Medoids')
        top_medoid_handle = mlines.Line2D([], [], color='purple', marker='D', linestyle='None',
                                        markersize=10, label='Top 5 Medoids')

        # Add legend
        plt.legend(handles=[max_sharpe_handle, medoid_handle, top_medoid_handle],
                loc='best')

        # Set labels and title
        plt.title('Hierarchical Clustering Analysis (Sharpe Ratio)', fontsize=14)
        plt.xlabel('Short SMA (days)', fontsize=12)
        plt.ylabel('Long SMA (days)', fontsize=12)

        # Rotate tick labels
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)

        # Display plot
        plt.tight_layout()
        save_plot('Hierarchical_Cluster_Analysis.png')

    # Main execution block
    # Main execution block
    if __name__ == "__main__":
            # Set matplotlib backend explicitly
            # Main execution block
            # Set matplotlib backend explicitly
            matplotlib.use('Agg')  # Use non-interactive backend for headless environments

            print("Starting ATR-based SMA strategy analysis...")

            # Run the basic analysis first
            data, best_short, best_long, best_sharpe, best_trades = analyze_sma_results()

            if data is None:
                print("Error: Failed to load or analyze SMA results data.")
                exit(1)

            print("\nProceeding with cluster analysis...")

            # Run the cluster analysis to get medoids
            X_filtered, medoids, top_medoids, max_sharpe_point, labels = hierarchical_cluster_analysis()

            if X_filtered is None or medoids is None:
                print("Error: Cluster analysis failed.")
                exit(1)

            # Re-sort top_medoids to ensure they're in the right order
            if top_medoids is not None:
                print("Re-sorting top medoids by Sharpe ratio...")
                # CRITICAL FIX: Use float() to ensure proper numeric comparison
                top_medoids = sorted(top_medoids, key=lambda x: float(x[2]), reverse=True)
                for idx, medoid in enumerate(top_medoids, 1):
                    print(f"Verified Medoid {idx}: Short SMA={int(medoid[0])}, Long SMA={int(medoid[1])}, "
                        f"Sharpe={float(medoid[2]):.4f}, Trades={int(medoid[3])}")

            print("\nPlotting strategy performance...")

            # Plot strategy performance with the best parameters AND top medoids using ATR-based position sizing
            market_data = plot_strategy_performance(
                best_short, best_long, top_medoids, 
                big_point_value=big_point_value,
                slippage=slippage,
                capital=capital,
                atr_period=atr_period
            )
            
            # Run the bimonthly out-of-sample comparison between best Sharpe and top medoids
            if top_medoids and len(top_medoids) > 0:
                print("\nPerforming bimonthly out-of-sample comparison with hierarchical clustering...")
                bimonthly_sharpe_df = bimonthly_out_of_sample_comparison(
                    market_data, 
                    best_short, 
                    best_long, 
                    top_medoids,  # Pass the entire list of top medoids
                    big_point_value=big_point_value,
                    slippage=slippage,
                    capital=capital,
                    atr_period=atr_period
                )
            else:
                print("No top medoids found. Cannot run bimonthly comparison.")
                
            print("\nAnalysis complete! All plots and result files have been saved to the output directory.")
            print(f"Output directory: {output_dir}")

# Call the main function to execute the script
if __name__ == "__main__":
    data_analysis_hierarchy_main()

## 7. Main Execution
This cell will run the complete analysis

In [ ]:
def run_analysis_for_ticker(ticker):
    global TICKER  # We'll modify the global TICKER variable
    TICKER = ticker  # Set the ticker for this run
    print(f"\n=== STARTING ANALYSIS FOR {TICKER} ===")
    
    try:
        # Run data gathering
        print("=== RUNNING DATA GATHERING ===")
        data_gather_main()
        
        # Run K-means analysis
        print("\n=== RUNNING K-MEANS ANALYSIS ===")
        data_analysis_main()
        
        # Run hierarchical analysis
        print("\n=== RUNNING HIERARCHICAL ANALYSIS ===")
        data_analysis_hierarchy_main()
        
        print(f"\nCompleted analysis for {TICKER}")
        return True
    except Exception as e:
        print(f"Error analyzing {TICKER}: {str(e)}")
        return False

def main(tickers=None):
    """
    Run analysis for specified tickers or all underlyings
    Args:
        tickers: List of tickers to analyze. If None, uses all underlyings
    """
    if tickers is None:
        tickers = underlyings
    elif isinstance(tickers, str):
        tickers = [tickers]  # Convert single ticker to list
    
    successful = []
    failed = []
    
    for ticker in tickers:
        print(f"\nProcessing {ticker}...")
        if run_analysis_for_ticker(ticker):
            successful.append(ticker)
        else:
            failed.append(ticker)
    
    # Print summary
    print("\n=== ANALYSIS SUMMARY ===")
    print(f"Successfully analyzed: {len(successful)} tickers")
    print(f"Failed to analyze: {len(failed)} tickers")
    if failed:
        print("Failed tickers:", failed)

if __name__ == "__main__":
    main()